# `ALL`: Collect bigrams corresponding to top adverbs

In [14]:
from pathlib import Path
from pprint import pprint

import pandas as pd
from am_notebooks import (load_bigram_dfs, locate_bigram_am_paths,
                          nb_show_table, show_adv_bigrams)

from source.utils.associate import TOP_AM_DIR, adjust_assoc_columns
from source.utils.general import confirm_dir, print_iter, timestamp_today

ADV_FLOOR = 5000
K = 8

DATA_DATE = '2024-07-28'
TAG = 'ALL'
TOP_AM_TAG_DIR = TOP_AM_DIR / TAG
confirm_dir(TOP_AM_TAG_DIR)

data_top = f'{TAG}-Top{K}'
OUT_DIR = TOP_AM_TAG_DIR / data_top
confirm_dir(OUT_DIR)

# for loading `polar/*/bigram/*` tables
bigram_floor = 25 
mirror_floor = 5 
if TAG=='ALL':
    bigram_floor, mirror_floor = [x*2 for x in (bigram_floor, mirror_floor)]

## Set columns and diplay settings

In [15]:
FOCUS = ['f',
         'am_p1_given2', 'am_p1_given2_simple', 
         'conservative_log_ratio',
         'am_log_likelihood',
          'mutual_information',
         'am_odds_ratio_disc', 't_score',
         'N', 'f1', 'f2', 'E11', 'unexpected_f',
         'l1', 'l2', 
         'adv', 'adv_total', 'adj', 'adj_total']
pd.set_option('display.max_colwidth', 30)
pd.set_option('display.max_columns', 9)
pd.set_option('display.width', 120)
pd.set_option("display.precision", 2)
pd.set_option("styler.format.precision", 2)
pd.set_option("styler.format.thousands", ",")
pd.set_option("display.float_format", '{:,.2f}'.format)

(MOVED the following to `./am_notebooks.py`)

In [16]:
# def force_ints(_df):
#     count_cols = _df.filter(regex=r'total|^[fN]').columns
#     _df[count_cols] = _df[count_cols].astype('int')
#     # _df[count_cols] = _df[:, count_cols].astype('int64')
#     # print(_df.dtypes.to_frame('dtypes'))
#     return _df


# def nb_show_table(df, n_dec: int = 2,
#                   adjust_columns: bool = True,
#                   outpath: Path = None,
#                   return_df: bool = False) -> None:
#     _df = df.copy()
#     try:
#         start_0 = _df.index.start == 0
#     except AttributeError:
#         pass
#     else:
#         _df.index.name = 'rank'
#         if start_0:
#             _df.index = _df.index + 1
#     if adjust_columns:
#         _df = adjust_assoc_columns(_df)
#     _df.columns = [f'`{c}`' for c in _df.columns]
#     _df.index = [f'**{r}**' for r in _df.index]
#     table = _df.to_markdown(floatfmt=f',.{n_dec}f', intfmt=',')
#     if outpath:
#         outpath.write_text(table)

#     print(f'\n{table}\n')
#     return (_df if return_df else None)
# from pprint import pprint
# from source.utils.dataframes import catify_hit_table as catify


# bigram_am_paths = {d.name:
              
#                   tuple(d.joinpath('bigram/extra').glob(
#                       f'*{TAG}*min{mirror_floor if d.name.endswith("mirror") else bigram_floor}x*extra.parq'))[0]
                  
#               for d in POLAR_DIR.iterdir()}

# pprint(bigram_am_paths)
# bigram_dfs = {n: catify(update_index(pd.read_parquet(p)), 
#                         reverse=True) 
#               for n, p in bigram_am_paths.items()}

# def show_adv_bigrams(sample_size, C, 
#                      bigram_dfs, 
#                      selector: str = 'dP1', 
#                      column_list: list = None) -> dict:
#     def _force_ints(_df):
#         count_cols = _df.filter(regex=r'total$|^[fN]').columns
#         _df.loc[:, count_cols] = _df.loc[:, count_cols].apply(
#             pd.to_numeric, downcast='unsigned')
#         return _df

#     def get_top_bigrams(bdf, adv, bigram_k):
#         bdf = _force_ints(bdf.loc[bdf.adv == adv, :])
#         top_by_metric = [bdf.nlargest(bigram_k * 2, m) for m in ['dP1', 'LRC']]
#         half_k = bigram_k // 2
#         adv_pat_bigrams = pd.concat(
#             [top_bigrams.head(half_k) for top_bigrams in top_by_metric]).drop_duplicates()
#         x = 0
#         while len(adv_pat_bigrams) < min(bigram_k, len(bdf)):
#             x += 1
#             next_ix = half_k + x
            
#             try:
#                 next_entries = [top_by_metric[0].iloc[[next_ix], :], 
#                             top_by_metric[1].iloc[[next_ix], :]]
#             except IndexError:
#                 print(f'All bigrams for {adv} retrieved.')
#                 break
#             else:
#                 adv_pat_bigrams = pd.concat((adv_pat_bigrams, 
#                                          *next_entries)).drop_duplicates()
#         return adv_pat_bigrams.head(bigram_k)

#     bigram_k = max(sample_size + 2, 10)
#     print(
#         f'## Top {bigram_k} "most negative" bigrams corresponding to top {sample_size} adverbs\n')
#     print(timestamp_today())
#     patterns = list(bigram_dfs.keys())
#     top_adverbs = C.index
#     bigram_samples = {adv: dict.fromkeys(
#         patterns + ['both', 'adj']) for adv in top_adverbs}
#     bigrams, adj = [], []

#     for rank, adv in enumerate(top_adverbs, start=1):
#         print(f'\n### {rank}. _{adv}_\n')
#         adj_for_adv = []
#         adv_top = None

#         for pat, bdf in bigram_dfs.items():
#             bdf = adjust_assoc_columns(
#                 bdf[[c for c in FOCUS + ['adj', 'adj_total', 'adv', 'adv_total'] if c in bdf.columns]])
#             bdf = bdf.loc[bdf.LRC >= 1, :]
#             adv_pat_bigrams = get_top_bigrams(bdf, adv, bigram_k)

#             if adv_pat_bigrams.empty:
#                 print(f'No bigrams found in loaded `{pat}` AM table.')
#             else:
#                 print(
#                     f'\n#### Top {len(adv_pat_bigrams)} `{pat}` "{adv}_*" bigrams (sorted by `{selector}`; `LRC > 1`)\n')
#                 column_list = column_list if column_list is not None else bdf.columns
#                 nb_show_table(adv_pat_bigrams.filter(column_list), n_dec=2)

#             adj_for_adv.extend(adv_pat_bigrams.adj.drop_duplicates().to_list())
#             bigram_samples[adv][pat] = adv_pat_bigrams
#             if adv_top is None: 
#                 adv_top = adv_pat_bigrams 
#             else:
#                 adv_top = pd.concat([df.fillna('') for df in (adv_top, adv_pat_bigrams)])

#         bigram_samples[adv]['adj'] = set(adj_for_adv)
#         bigrams.extend(adv_top.l2.drop_duplicates().to_list())
#         adj.extend(adj_for_adv)
#         bigram_samples[adv]['both'] = adv_top

#     bigram_samples['bigrams'] = set(bigrams)
#     bigram_samples['adj'] = set(adj)
#     return bigram_samples, bigram_k

## Load data

In [17]:
bigram_dfs = load_bigram_dfs(locate_bigram_am_paths(TAG, mirror_floor, bigram_floor))

try:
    combined_am_csv = tuple(OUT_DIR.glob(
                   f'{TAG}-Top{K}_NEG-ADV_combined-{ADV_FLOOR}*.{DATA_DATE or timestamp_today()}.csv'))[0]
except IndexError:
    combined_am_csv = tuple(TOP_AM_TAG_DIR.rglob(
        f'{TAG}-Top{K}_NEG-ADV_combined*.csv'))[0]
print(f'Loaded from: "{combined_am_csv}"')

C = adjust_assoc_columns(pd.read_csv(combined_am_csv, index_col='adv'))

main_cols_ordered = pd.concat((*[C.filter(like=m).columns.to_series() for m in ('LRC', 'P1', 'G2')],
                               *[C.filter(regex=f'^{f}_').columns.to_series() for f in ['f', 'f1', 'f2']])
                              ).to_list()
C

{'RBdirect': PosixPath('/share/compling/projects/sanpi/results/assoc_df/polar/RBdirect/bigram/extra/polarized-bigram_ALL-direct_min50x_extra.parq'),
 'mirror': PosixPath('/share/compling/projects/sanpi/results/assoc_df/polar/mirror/bigram/extra/polarized-bigram_ALL-mirror_min10x_extra.parq')}
Loaded from: "/share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8/ALL-Top8_NEG-ADV_combined-5000-500.2024-07-28.csv"


,key_SET,f_SET,dP1_SET,P1_SET,...,r_f_MIR,r_N_MIR,r_f1_MIR,r_f2_MIR
adv,,,,,,,,,
necessarily,NEGany~necessarily,42595,0.83,0.87,...,0.02,0.02,0.09,0.02
that,NEGany~that,164768,0.75,0.79,...,0.03,0.02,0.09,0.03
exactly,NEGany~exactly,43813,0.70,0.75,...,0.02,0.02,0.09,0.02
any,NEGany~any,15384,0.40,0.45,...,0.07,0.02,0.09,0.03
remotely,NEGany~remotely,5661,0.30,0.34,...,0.33,0.02,0.09,0.14
ever,NEGany~ever,5932,0.01,0.05,...,0.79,0.02,0.09,0.04
yet,NEGany~yet,51867,0.50,0.54,...,0.01,0.02,0.09,0.01
immediately,NEGany~immediately,56099,0.54,0.58,...,0.01,0.02,0.09,0.01
particularly,NEGany~particularly,55527,0.07,0.11,...,0.17,0.02,0.09,0.03


In [18]:
samples_dict, bigram_k = show_adv_bigrams(
    K, C, bigram_dfs,
    column_list=[
        'adj', 'adj_total',
        *pd.Series(main_cols_ordered).str.replace(
            r'mean_|_SET|_MIR', '', regex=True)
        .drop_duplicates().to_list(),
        # 't', 'MI'
    ], focus_cols=FOCUS
)

## Top 10 "most negative" bigrams corresponding to top 8 adverbs

2024-07-28

### 1. _necessarily_


#### Top 10 `RBdirect` "necessarily_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                       | `adj`          |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |
|:--------------------------------------|:---------------|--------------:|--------:|--------:|-------:|---------:|------:|----------:|-------:|
| **NEGany~necessarily_useful**         | useful         |       227,709 |    6.75 |    0.96 |   1.00 |   651.75 |   104 | 3,173,660 |    104 |
| **NEGany~necessarily_fun**            | fun            |       190,026 |    6.62 |    0.96 |   1.00 |   601.61 |    96 | 3,173,660 |     96 |
| **NEGany~necessarily_essential**      | essential      |        69,845 |    6.57 |    0.96 |   1.00 |   582.81 |    93 | 3,173,660 |     93 |
| **NEGany~necessarily_reliable**       | reliable       |        90,598 |    6.35 |    0.96 |   1.00

## Top 10 "most negative" bigrams corresponding to top 8 adverbs

2024-07-28

### 1. _necessarily_


#### Top 10 `RBdirect` "necessarily_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                       | `adj`          |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |
|:--------------------------------------|:---------------|--------------:|--------:|--------:|-------:|---------:|------:|----------:|-------:|
| **NEGany~necessarily_useful**         | useful         |       227,709 |    6.75 |    0.96 |   1.00 |   651.75 |   104 | 3,173,660 |    104 |
| **NEGany~necessarily_fun**            | fun            |       190,026 |    6.62 |    0.96 |   1.00 |   601.61 |    96 | 3,173,660 |     96 |
| **NEGany~necessarily_essential**      | essential      |        69,845 |    6.57 |    0.96 |   1.00 |   582.81 |    93 | 3,173,660 |     93 |
| **NEGany~necessarily_reliable**       | reliable       |        90,598 |    6.35 |    0.96 |   1.00 |   507.61 |    81 | 3,173,660 |     81 |
| **NEGany~necessarily_proud**          | proud          |       207,536 |    6.14 |    0.96 |   1.00 |   444.94 |    71 | 3,173,660 |     71 |
| **NEGany~necessarily_indicative**     | indicative     |         8,148 |    9.43 |    0.95 |   0.99 | 8,577.54 | 1,389 | 3,173,660 |  1,400 |
| **NEGany~necessarily_easy**           | easy           |       579,827 |    9.01 |    0.95 |   0.99 | 5,605.64 |   909 | 3,173,660 |    917 |
| **NEGany~necessarily_representative** | representative |        18,355 |    8.34 |    0.95 |   0.99 | 2,996.58 |   487 | 3,173,660 |    492 |
| **NEGany~necessarily_surprising**     | surprising     |        70,540 |    8.33 |    0.95 |   1.00 | 2,117.16 |   340 | 3,173,660 |    341 |
| **NEGany~necessarily_new**            | new            |       253,862 |    7.94 |    0.94 |   0.98 | 2,923.82 |   482 | 3,173,660 |    492 |


#### Top 7 `mirror` "necessarily_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                | `adj`   |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |
|:-------------------------------|:--------|--------------:|--------:|--------:|-------:|-------:|------:|--------:|-------:|
| **NEGmir~necessarily_right**   | right   |         5,576 |    2.15 |    0.83 |   1.00 |  81.13 |    23 | 291,732 |     23 |
| **NEGmir~necessarily_illegal** | illegal |           937 |    1.20 |    0.83 |   1.00 |  52.91 |    15 | 291,732 |     15 |
| **NEGmir~necessarily_wrong**   | wrong   |        20,880 |    5.04 |    0.81 |   0.98 | 698.74 |   211 | 291,732 |    216 |
| **NEGmir~necessarily_new**     | new     |        12,836 |    1.84 |    0.79 |   0.96 |  73.19 |    23 | 291,732 |     24 |
| **NEGmir~necessarily_bad**     | bad     |        10,261 |    2.95 |    0.77 |   0.94 | 154.45 |    50 | 291,732 |     53 |
| **NEGmir~necessarily_true**    | true    |         6,191 |    2.69 |    0.73 |   0.90 | 150.42 |    53 | 291,732 |     59 |
| **NEGmir~necessarily_better**  | better  |        14,013 |    1.63 |    0.70 |   0.87 |  72.90 |    27 | 291,732 |     31 |


### 2. _that_


#### Top 10 `RBdirect` "that_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                             | `adj`       |   `adj_total` |   `LRC` |   `dP1` |   `P1` |      `G2` |   `f` |      `f1` |   `f2` |
|:----------------------------|:------------|--------------:|--------:|--------:|-------:|----------:|------:|----------:|-------:|
| **NEGany~that_far-fetched** | far-fetched |         5,185 |    5.83 |    0.96 |   1.00 |    369.74 |    59 | 3,173,660 |     59 |
| **NEGany~that_thrilled**    | thrilled    |        24,182 |    5.83 |    0.96 |   1.00 |    369.74 |    59 | 3,173,660 |     59 |
| **NEGany~that_uncommon**    | uncommon    |        11,312 |    9.43 |    0.95 |   1.00 |  4,998.32 |   802 | 3,173,660 |    804 |
| **NEGany~that_surprising**  | surprising  |        70,540 |    9.20 |    0.95 |   0.99 |  6,986.81 | 1,133 | 3,173,660 |  1,143 |
| **NEGany~that_dissimilar**  | dissimilar  |         4,605 |    7.86 |    0.95 |   0.99 |  1,871.65 |   304 | 3,173,660 |    307 |
| **NEGany~that_unusual**     | unusual     |        71,234 |    8.92 |    0.95 |   0.99 |  6,003.05 |   977 | 3,173,660 |    988 |
| **NEGany~that_complicated** | complicated |       159,822 |    8.68 |    0.94 |   0.98 |  7,337.84 | 1,207 | 3,173,660 |  1,230 |
| **NEGany~that_hard**        | hard        |       348,463 |    8.59 |    0.91 |   0.96 | 58,817.24 | 9,948 | 3,173,660 | 10,380 |
| **NEGany~that_familiar**    | familiar    |       156,296 |    8.35 |    0.93 |   0.97 |  6,781.11 | 1,126 | 3,173,660 |  1,156 |
| **NEGany~that_noticeable**  | noticeable  |        31,467 |    7.65 |    0.95 |   0.99 |  1,621.81 |   264 | 3,173,660 |    267 |


#### Top 10 `mirror` "that_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                             | `adj`       |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |    `f1` |   `f2` |
|:----------------------------|:------------|--------------:|--------:|--------:|-------:|---------:|------:|--------:|-------:|
| **NEGmir~that_keen**        | keen        |         1,360 |    2.73 |    0.83 |   1.00 |   109.35 |    31 | 291,732 |     31 |
| **NEGmir~that_impressive**  | impressive  |         5,007 |    2.15 |    0.83 |   1.00 |    81.13 |    23 | 291,732 |     23 |
| **NEGmir~that_fond**        | fond        |         1,115 |    1.84 |    0.79 |   0.96 |    73.19 |    23 | 291,732 |     24 |
| **NEGmir~that_comfortable** | comfortable |         4,642 |    1.84 |    0.79 |   0.96 |    73.19 |    23 | 291,732 |     24 |
| **NEGmir~that_clear**       | clear       |         6,722 |    1.30 |    0.78 |   0.95 |    56.03 |    18 | 291,732 |     19 |
| **NEGmir~that_simple**      | simple      |        25,408 |    4.36 |    0.72 |   0.90 | 1,340.19 |   474 | 291,732 |    529 |
| **NEGmir~that_easy**        | easy        |        18,610 |    4.23 |    0.71 |   0.89 | 1,248.84 |   450 | 291,732 |    508 |
| **NEGmir~that_great**       | great       |         5,568 |    3.57 |    0.66 |   0.84 |   725.16 |   286 | 291,732 |    342 |
| **NEGmir~that_big**         | big         |         8,177 |    3.17 |    0.69 |   0.86 |   300.54 |   113 | 291,732 |    131 |
| **NEGmir~that_popular**     | popular     |         5,668 |    3.15 |    0.76 |   0.93 |   195.15 |    65 | 291,732 |     70 |


### 3. _exactly_


#### Top 10 `RBdirect` "exactly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                               | `adj`      |   `adj_total` |   `LRC` |   `dP1` |   `P1` |      `G2` |   `f` |      `f1` |   `f2` |
|:------------------------------|:-----------|--------------:|--------:|--------:|-------:|----------:|------:|----------:|-------:|
| **NEGany~exactly_conducive**  | conducive  |         9,110 |    7.82 |    0.96 |   1.00 |  1,303.50 |   208 | 3,173,660 |    208 |
| **NEGany~exactly_shocking**   | shocking   |        35,115 |    7.33 |    0.96 |   1.00 |    946.29 |   151 | 3,173,660 |    151 |
| **NEGany~exactly_pleasant**   | pleasant   |        52,223 |    7.24 |    0.96 |   1.00 |    889.88 |   142 | 3,173,660 |    142 |
| **NEGany~exactly_famous**     | famous     |       223,813 |    7.10 |    0.96 |   1.00 |    814.68 |   130 | 3,173,660 |    130 |
| **NEGany~exactly_difficult**  | difficult  |       732,106 |    7.05 |    0.96 |   1.00 |    789.62 |   126 | 3,173,660 |    126 |
| **NEGany~exactly_easy**       | easy       |       579,827 |    9.32 |    0.95 |   0.99 |  6,596.91 | 1,066 | 3,173,660 |  1,073 |
| **NEGany~exactly_new**        | new        |       253,862 |    9.10 |    0.94 |   0.99 |  8,410.32 | 1,371 | 3,173,660 |  1,388 |
| **NEGany~exactly_cheap**      | cheap      |        60,531 |    8.96 |    0.95 |   0.99 |  4,281.59 |   691 | 3,173,660 |    695 |
| **NEGany~exactly_clear**      | clear      |       349,214 |    8.78 |    0.94 |   0.98 | 10,578.33 | 1,746 | 3,173,660 |  1,784 |
| **NEGany~exactly_surprising** | surprising |        70,540 |    8.71 |    0.95 |   1.00 |  2,743.34 |   440 | 3,173,660 |    441 |


#### Top 4 `mirror` "exactly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                          | `adj`   |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |
|:-------------------------|:--------|--------------:|--------:|--------:|-------:|-------:|------:|--------:|-------:|
| **NEGmir~exactly_sure**  | sure    |         6,761 |    5.31 |    0.83 |   1.00 | 522.11 |   148 | 291,732 |    148 |
| **NEGmir~exactly_easy**  | easy    |        18,610 |    1.86 |    0.83 |   1.00 |  70.55 |    20 | 291,732 |     20 |
| **NEGmir~exactly_clear** | clear   |         6,722 |    3.38 |    0.81 |   0.98 | 173.89 |    52 | 291,732 |     53 |
| **NEGmir~exactly_new**   | new     |        12,836 |    2.31 |    0.80 |   0.97 |  93.90 |    29 | 291,732 |     30 |


### 4. _any_


#### Top 10 `RBdirect` "any_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                         | `adj`    |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |
|:------------------------|:---------|--------------:|--------:|--------:|-------:|---------:|------:|----------:|-------:|
| **NEGany~any_happier**  | happier  |        16,606 |    6.34 |    0.82 |   0.86 | 4,420.49 |   828 | 3,173,660 |    963 |
| **NEGany~any_younger**  | younger  |        26,216 |    5.57 |    0.79 |   0.83 | 1,323.36 |   255 | 3,173,660 |    307 |
| **NEGany~any_nicer**    | nicer    |         9,955 |    4.75 |    0.76 |   0.81 |   486.82 |    96 | 3,173,660 |    119 |
| **NEGany~any_simpler**  | simpler  |        23,480 |    5.00 |    0.71 |   0.75 | 1,087.71 |   226 | 3,173,660 |    300 |
| **NEGany~any_brighter** | brighter |         9,280 |    3.91 |    0.67 |   0.72 |   292.00 |    63 | 3,173,660 |     88 |
| **NEGany~any_easier**   | easier   |       209,940 |    5.08 |    0.62 |   0.66 | 6,987.78 | 1,594 | 3,173,660 |  2,405 |
| **NEGany~any_cheaper**  | cheaper  |        46,055 |    4.01 |    0.58 |   0.63 |   542.97 |   129 | 3,173,660 |    206 |
| **NEGany~any_smarter**  | smarter  |         8,501 |    4.00 |    0.63 |   0.67 |   394.95 |    89 | 3,173,660 |    132 |
| **NEGany~any_clearer**  | clearer  |        11,680 |    4.26 |    0.54 |   0.58 | 1,421.60 |   355 | 3,173,660 |    608 |
| **NEGany~any_worse**    | worse    |       179,012 |    3.94 |    0.42 |   0.46 | 5,676.37 | 1,686 | 3,173,660 |  3,673 |


#### Top 10 `mirror` "any_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                          | `adj`     |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |    `f1` |   `f2` |
|:-------------------------|:----------|--------------:|--------:|--------:|-------:|---------:|------:|--------:|-------:|
| **NEGmir~any_younger**   | younger   |           939 |    1.86 |    0.83 |   1.00 |    70.55 |    20 | 291,732 |     20 |
| **NEGmir~any_clearer**   | clearer   |           130 |    1.50 |    0.83 |   1.00 |    59.97 |    17 | 291,732 |     17 |
| **NEGmir~any_different** | different |        36,166 |    3.24 |    0.81 |   0.98 |   159.93 |    48 | 291,732 |     49 |
| **NEGmir~any_bigger**    | bigger    |         3,923 |    2.73 |    0.80 |   0.97 |   118.17 |    36 | 291,732 |     37 |
| **NEGmir~any_easier**    | easier    |         2,386 |    3.04 |    0.75 |   0.92 |   181.65 |    61 | 291,732 |     66 |
| **NEGmir~any_better**    | better    |        14,013 |    4.38 |    0.74 |   0.91 | 1,096.01 |   380 | 291,732 |    419 |
| **NEGmir~any_worse**     | worse     |         8,790 |    2.73 |    0.66 |   0.83 |   217.46 |    87 | 291,732 |    105 |
| **NEGmir~any_higher**    | higher    |         2,893 |    1.42 |    0.74 |   0.91 |    61.24 |    21 | 291,732 |     23 |
| **NEGmir~any_closer**    | closer    |           993 |    2.33 |    0.65 |   0.83 |   141.82 |    57 | 291,732 |     69 |
| **NEGmir~any_good**      | good      |        31,585 |    2.12 |    0.74 |   0.91 |    93.53 |    32 | 291,732 |     35 |


### 5. _remotely_


#### Top 10 `RBdirect` "remotely_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                | `adj`      |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |
|:-------------------------------|:-----------|--------------:|--------:|--------:|-------:|---------:|------:|----------:|-------:|
| **NEGany~remotely_surprising** | surprising |        70,540 |    5.07 |    0.85 |   0.89 |   413.61 |    75 | 3,173,660 |     84 |
| **NEGany~remotely_ready**      | ready      |       141,590 |    4.16 |    0.74 |   0.78 |   287.63 |    58 | 3,173,660 |     74 |
| **NEGany~remotely_true**       | true       |       231,639 |    4.61 |    0.63 |   0.68 | 1,111.13 |   250 | 3,173,660 |    370 |
| **NEGany~remotely_funny**      | funny      |       122,927 |    4.15 |    0.60 |   0.65 |   589.74 |   137 | 3,173,660 |    212 |
| **NEGany~remotely_qualified**  | qualified  |        74,643 |    3.13 |    0.52 |   0.56 |   222.79 |    57 | 3,173,660 |    101 |
| **NEGany~remotely_close**      | close      |       411,329 |    3.65 |    0.39 |   0.43 | 2,243.22 |   694 | 3,173,660 |  1,597 |
| **NEGany~remotely_comparable** | comparable |        12,252 |    2.98 |    0.38 |   0.43 |   375.73 |   118 | 3,173,660 |    277 |
| **NEGany~remotely_accurate**   | accurate   |       152,299 |    2.10 |    0.33 |   0.37 |   143.78 |    50 | 3,173,660 |    134 |
| **NEGany~remotely_interested** | interested |       264,528 |    2.74 |    0.27 |   0.31 |   817.06 |   330 | 3,173,660 |  1,062 |
| **NEGany~remotely_similar**    | similar    |       203,453 |    2.20 |    0.23 |   0.27 |   334.61 |   152 | 3,173,660 |    559 |


#### Top 10 `mirror` "remotely_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                 | `adj`       |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |
|:--------------------------------|:------------|--------------:|--------:|--------:|-------:|-------:|------:|--------:|-------:|
| **NEGmir~remotely_believable**  | believable  |           600 |    1.96 |    0.83 |   1.00 |  74.08 |    21 | 291,732 |     21 |
| **NEGmir~remotely_surprising**  | surprising  |         2,662 |    1.50 |    0.83 |   1.00 |  59.97 |    17 | 291,732 |     17 |
| **NEGmir~remotely_comparable**  | comparable  |           283 |    2.72 |    0.76 |   0.94 | 134.02 |    44 | 291,732 |     47 |
| **NEGmir~remotely_true**        | true        |         6,191 |    3.04 |    0.75 |   0.92 | 181.65 |    61 | 291,732 |     66 |
| **NEGmir~remotely_new**         | new         |        12,836 |    1.00 |    0.69 |   0.86 |  50.62 |    19 | 291,732 |     22 |
| **NEGmir~remotely_close**       | close       |        13,874 |    3.28 |    0.65 |   0.82 | 532.96 |   218 | 291,732 |    267 |
| **NEGmir~remotely_funny**       | funny       |         5,365 |    1.85 |    0.63 |   0.80 |  97.91 |    41 | 291,732 |     51 |
| **NEGmir~remotely_similar**     | similar     |         7,011 |    1.70 |    0.49 |   0.66 | 127.32 |    71 | 291,732 |    107 |
| **NEGmir~remotely_possible**    | possible    |         3,160 |    1.42 |    0.56 |   0.73 |  78.73 |    38 | 291,732 |     52 |
| **NEGmir~remotely_interesting** | interesting |        12,447 |    1.80 |    0.56 |   0.73 | 115.20 |    56 | 291,732 |     77 |


### 6. _ever_


#### Top 10 `RBdirect` "ever_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                           | `adj`     |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |       `f1` |   `f2` |
|:--------------------------|:----------|--------------:|--------:|--------:|-------:|---------:|------:|-----------:|-------:|
| **NEGany~ever_simple**    | simple    |       396,749 |    5.56 |    0.80 |   0.84 | 1,109.28 |   211 |  3,173,660 |    250 |
| **NEGany~ever_boring**    | boring    |        45,891 |    4.46 |    0.76 |   0.80 |   362.74 |    72 |  3,173,660 |     90 |
| **NEGany~ever_easy**      | easy      |       579,827 |    5.41 |    0.73 |   0.77 | 2,105.13 |   429 |  3,173,660 |    555 |
| **NEGany~ever_certain**   | certain   |        74,952 |    4.80 |    0.72 |   0.76 |   713.34 |   147 |  3,173,660 |    193 |
| **NEGany~ever_sure**      | sure      |       262,825 |    3.34 |    0.49 |   0.54 |   328.20 |    87 |  3,173,660 |    162 |
| **NEGany~ever_good**      | good      |     1,681,795 |    3.81 |    0.46 |   0.50 | 1,188.69 |   331 |  3,173,660 |    660 |
| **COM~ever_closer**       | closer    |        61,475 |    3.51 |    0.04 |   1.00 |   538.66 | 6,305 | 69,662,736 |  6,307 |
| **NEGany~ever_enough**    | enough    |       152,020 |    3.54 |    0.45 |   0.50 |   618.62 |   173 |  3,173,660 |    347 |
| **NEGany~ever_perfect**   | perfect   |       104,659 |    3.34 |    0.40 |   0.44 |   706.71 |   216 |  3,173,660 |    489 |
| **NEGany~ever_satisfied** | satisfied |        62,862 |    2.57 |    0.38 |   0.42 |   203.01 |    64 |  3,173,660 |    151 |


#### Top 10 `mirror` "ever_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                         | `adj`   |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |    `f1` |   `f2` |
|:------------------------|:--------|--------------:|--------:|--------:|-------:|---------:|------:|--------:|-------:|
| **NEGmir~ever_simple**  | simple  |        25,408 |    5.82 |    0.83 |   1.00 |   726.76 |   206 | 291,732 |    206 |
| **NEGmir~ever_enough**  | enough  |         2,596 |    5.30 |    0.83 |   1.00 |   518.58 |   147 | 291,732 |    147 |
| **NEGmir~ever_certain** | certain |         1,800 |    5.26 |    0.83 |   1.00 |   504.47 |   143 | 291,732 |    143 |
| **NEGmir~ever_boring**  | boring  |         1,961 |    3.80 |    0.83 |   1.00 |   201.07 |    57 | 291,732 |     57 |
| **NEGmir~ever_black**   | black   |         1,412 |    3.77 |    0.83 |   1.00 |   197.54 |    56 | 291,732 |     56 |
| **NEGmir~ever_easy**    | easy    |        18,610 |    6.44 |    0.83 |   1.00 | 1,285.01 |   368 | 291,732 |    369 |
| **NEGmir~ever_good**    | good    |        31,585 |    5.68 |    0.82 |   0.99 | 1,013.87 |   299 | 291,732 |    303 |
| **NEGmir~ever_perfect** | perfect |         3,134 |    5.57 |    0.82 |   1.00 |   714.47 |   206 | 291,732 |    207 |
| **NEGmir~ever_sick**    | sick    |         1,895 |    3.15 |    0.83 |   1.00 |   137.57 |    39 | 291,732 |     39 |
| **NEGmir~ever_able**    | able    |         3,704 |    4.17 |    0.78 |   0.95 |   426.52 |   136 | 291,732 |    143 |


### 7. _yet_


#### Top 10 `RBdirect` "yet_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                           | `adj`      |   `adj_total` |   `LRC` |   `dP1` |   `P1` |      `G2` |    `f` |      `f1` |   `f2` |
|:--------------------------|:-----------|--------------:|--------:|--------:|-------:|----------:|-------:|----------:|-------:|
| **NEGany~yet_eligible**   | eligible   |        23,252 |    8.96 |    0.96 |   1.00 |  2,807.56 |    448 | 3,173,660 |    448 |
| **NEGany~yet_official**   | official   |         6,853 |    8.61 |    0.96 |   1.00 |  2,205.93 |    352 | 3,173,660 |    352 |
| **NEGany~yet_convinced**  | convinced  |        12,132 |    7.50 |    0.96 |   1.00 |  1,059.09 |    169 | 3,173,660 |    169 |
| **NEGany~yet_online**     | online     |        15,650 |    6.66 |    0.96 |   1.00 |    614.14 |     98 | 3,173,660 |     98 |
| **NEGany~yet_mainstream** | mainstream |        17,792 |    6.11 |    0.96 |   1.00 |    438.67 |     70 | 3,173,660 |     70 |
| **NEGany~yet_clear**      | clear      |       349,214 |   10.77 |    0.95 |   0.99 | 64,409.97 | 10,406 | 3,173,660 | 10,476 |
| **NEGany~yet_ready**      | ready      |       141,590 |    9.93 |    0.94 |   0.99 | 45,985.07 |  7,501 | 3,173,660 |  7,599 |
| **NEGany~yet_complete**   | complete   |        86,361 |    9.20 |    0.94 |   0.98 | 13,277.09 |  2,174 | 3,173,660 |  2,208 |
| **NEGany~yet_final**      | final      |         5,860 |    8.97 |    0.95 |   1.00 |  3,972.92 |    640 | 3,173,660 |    643 |
| **NEGany~yet_over**       | over       |         3,983 |    7.21 |    0.95 |   0.99 |  1,003.13 |    162 | 3,173,660 |    163 |


#### Top 5 `mirror` "yet_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                          | `adj`     |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |
|:-------------------------|:----------|--------------:|--------:|--------:|-------:|-------:|------:|--------:|-------:|
| **NEGmir~yet_available** | available |        10,284 |    2.54 |    0.83 |   1.00 |  98.77 |    28 | 291,732 |     28 |
| **NEGmir~yet_certain**   | certain   |         1,800 |    1.96 |    0.83 |   1.00 |  74.08 |    21 | 291,732 |     21 |
| **NEGmir~yet_sure**      | sure      |         6,761 |    1.75 |    0.83 |   1.00 |  67.02 |    19 | 291,732 |     19 |
| **NEGmir~yet_clear**     | clear     |         6,722 |    1.75 |    0.83 |   1.00 |  67.02 |    19 | 291,732 |     19 |
| **NEGmir~yet_ready**     | ready     |         3,034 |    1.63 |    0.83 |   1.00 |  63.49 |    18 | 291,732 |     18 |


### 8. _immediately_


#### Top 10 `RBdirect` "immediately_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                    | `adj`       |   `adj_total` |   `LRC` |   `dP1` |   `P1` |       `G2` |    `f` |      `f1` |   `f2` |
|:-----------------------------------|:------------|--------------:|--------:|--------:|-------:|-----------:|-------:|----------:|-------:|
| **NEGany~immediately_possible**    | possible    |       245,272 |    7.62 |    0.91 |   0.95 |   5,845.77 |  1,000 | 3,173,660 |  1,054 |
| **NEGany~immediately_clear**       | clear       |       349,214 |    8.16 |    0.89 |   0.94 | 141,186.69 | 24,416 | 3,173,660 | 26,038 |
| **NEGany~immediately_reachable**   | reachable   |         2,672 |    5.56 |    0.86 |   0.91 |     610.53 |    109 | 3,173,660 |    120 |
| **NEGany~immediately_sure**        | sure        |       262,825 |    5.40 |    0.82 |   0.86 |     738.65 |    138 | 3,173,660 |    160 |
| **NEGany~immediately_certain**     | certain     |        74,952 |    4.19 |    0.71 |   0.75 |     336.68 |     70 | 3,173,660 |     93 |
| **NEGany~immediately_available**   | available   |       666,909 |    5.70 |    0.67 |   0.72 |  98,046.67 | 21,078 | 3,173,660 | 29,351 |
| **NEGany~immediately_able**        | able        |       223,196 |    4.70 |    0.59 |   0.63 |   2,655.18 |    626 | 3,173,660 |    989 |
| **NEGany~immediately_obvious**     | obvious     |       165,439 |    4.55 |    0.52 |   0.56 |   8,712.07 |  2,238 | 3,173,660 |  3,989 |
| **NEGany~immediately_forthcoming** | forthcoming |         7,473 |    3.89 |    0.55 |   0.60 |     540.70 |    133 | 3,173,660 |    223 |
| **NEGany~immediately_intuitive**   | intuitive   |        20,664 |    3.21 |    0.55 |   0.59 |     218.74 |     54 | 3,173,660 |     91 |


#### Top 2 `mirror` "immediately_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                  | `adj`     |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |
|:---------------------------------|:----------|--------------:|--------:|--------:|-------:|-------:|------:|--------:|-------:|
| **NEGmir~immediately_clear**     | clear     |         6,722 |    1.13 |    0.55 |   0.72 |  62.95 |    31 | 291,732 |     43 |
| **NEGmir~immediately_available** | available |        10,284 |    1.85 |    0.42 |   0.59 | 241.53 |   162 | 291,732 |    275 |


### 9. _particularly_


#### Top 10 `RBdirect` "particularly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                       | `adj`         |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |
|:--------------------------------------|:--------------|--------------:|--------:|--------:|-------:|---------:|------:|----------:|-------:|
| **NEGany~particularly_likable**       | likable       |         8,160 |    5.53 |    0.85 |   0.89 |   657.49 |   119 | 3,173,660 |    133 |
| **NEGany~particularly_likeable**      | likeable      |         5,902 |    5.34 |    0.84 |   0.88 |   579.07 |   106 | 3,173,660 |    120 |
| **NEGany~particularly_forthcoming**   | forthcoming   |         7,473 |    4.85 |    0.82 |   0.87 |   387.25 |    72 | 3,173,660 |     83 |
| **NEGany~particularly_religious**     | religious     |        28,028 |    6.12 |    0.81 |   0.86 | 2,585.71 |   485 | 3,173,660 |    565 |
| **NEGany~particularly_original**      | original      |        37,594 |    5.86 |    0.80 |   0.85 | 1,894.60 |   360 | 3,173,660 |    426 |
| **NEGany~particularly_new**           | new           |       253,862 |    6.04 |    0.79 |   0.83 | 3,874.46 |   747 | 3,173,660 |    900 |
| **NEGany~particularly_athletic**      | athletic      |        17,142 |    4.77 |    0.75 |   0.79 |   541.01 |   108 | 3,173,660 |    136 |
| **NEGany~particularly_revolutionary** | revolutionary |        10,338 |    4.49 |    0.75 |   0.79 |   385.60 |    77 | 3,173,660 |     97 |
| **NEGany~particularly_flashy**        | flashy        |         4,494 |    4.08 |    0.73 |   0.77 |   278.96 |    57 | 3,173,660 |     74 |
| **NEGany~particularly_surprising**    | surprising    |        70,540 |    4.71 |    0.57 |   0.61 | 4,433.52 | 1,069 | 3,173,660 |  1,743 |


#### Top 10 `mirror` "particularly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                       | `adj`         |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |    `f1` |   `f2` |
|:--------------------------------------|:--------------|--------------:|--------:|--------:|-------:|---------:|------:|--------:|-------:|
| **NEGmir~particularly_novel**         | novel         |           320 |    3.71 |    0.83 |   1.00 |   190.49 |    54 | 291,732 |     54 |
| **NEGmir~particularly_comfortable**   | comfortable   |         4,642 |    3.36 |    0.83 |   1.00 |   155.21 |    44 | 291,732 |     44 |
| **NEGmir~particularly_revolutionary** | revolutionary |           485 |    3.15 |    0.83 |   1.00 |   137.57 |    39 | 291,732 |     39 |
| **NEGmir~particularly_radical**       | radical       |         1,072 |    2.73 |    0.83 |   1.00 |   109.35 |    31 | 291,732 |     31 |
| **NEGmir~particularly_fast**          | fast          |         1,259 |    2.39 |    0.83 |   1.00 |    91.71 |    26 | 291,732 |     26 |
| **NEGmir~particularly_new**           | new           |        12,836 |    5.92 |    0.81 |   0.99 | 1,365.17 |   404 | 291,732 |    410 |
| **NEGmir~particularly_surprising**    | surprising    |         2,662 |    5.06 |    0.82 |   0.99 |   564.67 |   166 | 291,732 |    168 |
| **NEGmir~particularly_wrong**         | wrong         |        20,880 |    5.05 |    0.81 |   0.98 |   702.22 |   212 | 291,732 |    217 |
| **NEGmir~particularly_remarkable**    | remarkable    |         3,238 |    4.24 |    0.80 |   0.97 |   354.53 |   108 | 291,732 |    111 |
| **NEGmir~particularly_close**         | close         |        13,874 |    4.01 |    0.77 |   0.94 |   415.70 |   136 | 291,732 |    145 |


### 10. _inherently_

All bigrams for inherently retrieved.

#### Top 7 `RBdirect` "inherently_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                   | `adj`       |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |
|:----------------------------------|:------------|--------------:|--------:|--------:|-------:|---------:|------:|----------:|-------:|
| **NEGany~inherently_wrong**       | wrong       |       149,064 |    5.36 |    0.66 |   0.71 | 7,535.97 | 1,639 | 3,173,660 |  2,315 |
| **NEGany~inherently_illegal**     | illegal     |        30,194 |    3.51 |    0.60 |   0.64 |   252.59 |    59 | 3,173,660 |     92 |
| **NEGany~inherently_bad**         | bad         |       429,537 |    4.62 |    0.56 |   0.61 | 3,270.68 |   794 | 3,173,660 |  1,307 |
| **NEGany~inherently_negative**    | negative    |        53,385 |    2.50 |    0.35 |   0.39 |   222.63 |    75 | 3,173,660 |    193 |
| **NEGany~inherently_evil**        | evil        |        22,706 |    2.81 |    0.28 |   0.32 |   905.80 |   358 | 3,173,660 |  1,123 |
| **NEGany~inherently_good**        | good        |     1,681,795 |    2.21 |    0.20 |   0.24 |   554.72 |   283 | 3,173,660 |  1,177 |
| **NEGany~inherently_problematic** | problematic |        33,408 |    1.17 |    0.17 |   0.21 |    98.70 |    58 | 3,173,660 |    277 |

All bigrams for inherently retrieved.

#### Top 6 `mirror` "inherently_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                | `adj`    |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |    `f1` |   `f2` |
|:-------------------------------|:---------|--------------:|--------:|--------:|-------:|---------:|------:|--------:|-------:|
| **NEGmir~inherently_improper** | improper |           142 |    1.63 |    0.83 |   1.00 |    63.49 |    18 | 291,732 |     18 |
| **NEGmir~inherently_illegal**  | illegal  |           937 |    2.10 |    0.79 |   0.96 |    83.54 |    26 | 291,732 |     27 |
| **NEGmir~inherently_wrong**    | wrong    |        20,880 |    4.08 |    0.66 |   0.83 | 3,787.53 | 1,513 | 291,732 |  1,826 |
| **NEGmir~inherently_bad**      | bad      |        10,261 |    2.86 |    0.62 |   0.79 |   342.53 |   148 | 291,732 |    188 |
| **NEGmir~inherently_better**   | better   |        14,013 |    1.65 |    0.57 |   0.75 |    93.95 |    44 | 291,732 |     59 |
| **NEGmir~inherently_evil**     | evil     |         1,271 |    1.18 |    0.41 |   0.59 |    85.70 |    58 | 291,732 |     99 |


### 11. _terribly_


#### Top 10 `RBdirect` "terribly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                | `adj`      |   `adj_total` |   `LRC` |   `dP1` |   `P1` |     `G2` |   `f` |      `f1` |   `f2` |
|:-------------------------------|:-----------|--------------:|--------:|--------:|-------:|---------:|------:|----------:|-------:|
| **NEGany~terribly_surprising** | surprising |        70,540 |    8.66 |    0.94 |   0.98 | 5,794.10 |   949 | 3,173,660 |    964 |
| **NEGany~terribly_uncommon**   | uncommon   |        11,312 |    6.33 |    0.94 |   0.98 |   625.85 |   103 | 3,173,660 |    105 |
| **NEGany~terribly_likely**     | likely     |       890,421 |    6.26 |    0.93 |   0.97 |   649.50 |   108 | 3,173,660 |    111 |
| **NEGany~terribly_productive** | productive |       102,361 |    5.39 |    0.91 |   0.96 |   376.84 |    64 | 3,173,660 |     67 |
| **NEGany~terribly_reliable**   | reliable   |        90,598 |    4.99 |    0.90 |   0.94 |   296.70 |    51 | 3,173,660 |     54 |
| **NEGany~terribly_original**   | original   |        37,594 |    6.41 |    0.90 |   0.94 | 1,150.48 |   199 | 3,173,660 |    212 |
| **NEGany~terribly_different**  | different  |       825,838 |    6.33 |    0.85 |   0.89 | 2,022.47 |   366 | 3,173,660 |    409 |
| **NEGany~terribly_unusual**    | unusual    |        71,234 |    6.17 |    0.90 |   0.94 |   847.05 |   146 | 3,173,660 |    155 |
| **NEGany~terribly_sure**       | sure       |       262,825 |    5.08 |    0.86 |   0.91 |   386.31 |    69 | 3,173,660 |     76 |
| **NEGany~terribly_impressive** | impressive |       178,051 |    5.97 |    0.86 |   0.90 | 1,087.64 |   196 | 3,173,660 |    218 |


#### Top 10 `mirror` "terribly_*" bigrams (sorted by `dP1`; `LRC > 1`)


|                                 | `adj`       |   `adj_total` |   `LRC` |   `dP1` |   `P1` |   `G2` |   `f` |    `f1` |   `f2` |
|:--------------------------------|:------------|--------------:|--------:|--------:|-------:|-------:|------:|--------:|-------:|
| **NEGmir~terribly_surprising**  | surprising  |         2,662 |    4.07 |    0.83 |   1.00 | 236.35 |    67 | 291,732 |     67 |
| **NEGmir~terribly_original**    | original    |         1,555 |    3.40 |    0.83 |   1.00 | 158.74 |    45 | 291,732 |     45 |
| **NEGmir~terribly_special**     | special     |        15,541 |    2.23 |    0.83 |   1.00 |  84.66 |    24 | 291,732 |     24 |
| **NEGmir~terribly_unusual**     | unusual     |         2,302 |    2.23 |    0.83 |   1.00 |  84.66 |    24 | 291,732 |     24 |
| **NEGmir~terribly_popular**     | popular     |         5,668 |    1.75 |    0.83 |   1.00 |  67.02 |    19 | 291,732 |     19 |
| **NEGmir~terribly_new**         | new         |        12,836 |    3.66 |    0.80 |   0.97 | 225.93 |    69 | 291,732 |     71 |
| **NEGmir~terribly_interesting** | interesting |        12,447 |    2.44 |    0.68 |   0.85 | 145.16 |    56 | 291,732 |     66 |
| **NEGmir~terribly_interested**  | interested  |         8,255 |    2.36 |    0.74 |   0.91 | 112.46 |    39 | 291,732 |     43 |
| **NEGmir~terribly_clear**       | clear       |         6,722 |    1.20 |    0.83 |   1.00 |  52.91 |    15 | 291,732 |     15 |
| **NEGmir~terribly_remarkable**  | remarkable  |         3,238 |    1.03 |    0.83 |   1.00 |  49.38 |    14 | 291,732 |     14 |



 ### `2024-05-23` Top 10 "most negative" bigrams corresponding to top 5 adverbs



#### 1. _necessarily_


Top 10 `RBdirect` "necessarily_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                                       | `adj`          |   `adj_total` |   `LRC` |   `dP1` |      `G2` |   `f` |      `f1` |   `f2` |
 |:--------------------------------------|:---------------|--------------:|--------:|--------:|----------:|------:|----------:|-------:|
 | **NEGany~necessarily_sure**           | sure           |    844,981.00 |    5.91 |    0.95 |  1,436.68 |   222 | 3,226,213 |    224 |
 | **NEGany~necessarily_surprising**     | surprising     |    150,067.00 |    7.22 |    0.93 |  2,150.86 |   343 | 3,226,213 |    355 |
 | **NEGany~necessarily_indicative**     | indicative     |     12,760.00 |    8.37 |    0.93 |  8,811.69 | 1,406 | 3,226,213 |  1,456 |
 | **NEGany~necessarily_representative** | representative |     25,187.00 |    7.31 |    0.91 |  3,044.27 |   496 | 3,226,213 |    524 |
 | **NEGany~necessarily_available**      | available      |    866,272.00 |    6.36 |    0.89 |  1,280.24 |   213 | 3,226,213 |    230 |
 | **NEGany~necessarily_easy**           | easy           |    771,307.00 |    7.26 |    0.88 |  5,448.34 |   914 | 3,226,213 |    996 |
 | **NEGany~necessarily_true**           | true           |    348,994.00 |    6.89 |    0.82 | 18,199.76 | 3,238 | 3,226,213 |  3,786 |
 | **NEGany~necessarily_illegal**        | illegal        |     44,028.00 |    6.48 |    0.87 |  1,659.90 |   280 | 3,226,213 |    307 |
 | **NEGany~necessarily_related**        | related        |    137,661.00 |    6.74 |    0.84 |  4,271.76 |   742 | 3,226,213 |    842 |
 | **NEGany~necessarily_interested**     | interested     |    364,497.00 |    6.77 |    0.87 |  2,500.26 |   422 | 3,226,213 |    463 |


Top 3 `NEGmirror` "necessarily_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                              | `adj`   |   `adj_total` |   `LRC` |   `dP1` |   `G2` |   `f` |    `f1` |   `f2` |
 |:-----------------------------|:--------|--------------:|--------:|--------:|-------:|------:|--------:|-------:|
 | **NEGmir~necessarily_wrong** | wrong   |     20,866.00 |    4.27 |    0.81 | 708.98 |   209 | 289,770 |    214 |
 | **NEGmir~necessarily_bad**   | bad     |     10,783.00 |    2.02 |    0.76 | 153.43 |    50 | 289,770 |     54 |
 | **NEGmir~necessarily_true**  | true    |      7,402.00 |    2.18 |    0.75 | 159.07 |    53 | 289,770 |     58 |


#### 2. _exactly_


Top 10 `RBdirect` "exactly_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                               | `adj`      |   `adj_total` |   `LRC` |   `dP1` |      `G2` |   `f` |      `f1` |   `f2` |
 |:------------------------------|:-----------|--------------:|--------:|--------:|----------:|------:|----------:|-------:|
 | **NEGany~exactly_surprising** | surprising |    150,067.00 |    7.34 |    0.96 |  2,863.35 |   441 | 3,226,213 |    444 |
 | **NEGany~exactly_cheap**      | cheap      |     83,765.00 |    8.28 |    0.95 |  4,443.27 |   693 | 3,226,213 |    704 |
 | **NEGany~exactly_subtle**     | subtle     |     56,845.00 |    6.92 |    0.94 |  1,671.02 |   264 | 3,226,213 |    271 |
 | **NEGany~exactly_fun**        | fun        |    224,457.00 |    6.67 |    0.94 |  1,423.92 |   225 | 3,226,213 |    231 |
 | **NEGany~exactly_conducive**  | conducive  |     16,405.00 |    6.56 |    0.93 |  1,313.09 |   208 | 3,226,213 |    214 |
 | **NEGany~exactly_sure**       | sure       |    844,981.00 |    8.63 |    0.92 | 54,750.58 | 8,860 | 3,226,213 |  9,301 |
 | **NEGany~exactly_new**        | new        |    321,311.00 |    8.54 |    0.93 |  8,697.93 | 1,378 | 3,226,213 |  1,418 |
 | **NEGany~exactly_easy**       | easy       |    771,307.00 |    8.37 |    0.93 |  6,747.64 | 1,069 | 3,226,213 |  1,100 |
 | **NEGany~exactly_clear**      | clear      |    491,108.00 |    8.30 |    0.92 | 10,937.16 | 1,759 | 3,226,213 |  1,835 |
 | **NEGany~exactly_happy**      | happy      |    528,511.00 |    7.16 |    0.90 |  2,694.69 |   441 | 3,226,213 |    468 |

 No bigrams found in loaded `NEGmirror` AM table.

#### 3. _that_


Top 10 `RBdirect` "that_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                             | `adj`       |   `adj_total` |   `LRC` |   `dP1` |      `G2` |   `f` |      `f1` |   `f2` |
 |:----------------------------|:------------|--------------:|--------:|--------:|----------:|------:|----------:|-------:|
 | **NEGany~that_uncommon**    | uncommon    |     61,767.00 |    8.39 |    0.94 |  5,136.91 |   804 | 3,226,213 |    819 |
 | **NEGany~that_fond**        | fond        |     39,809.00 |    7.27 |    0.94 |  2,127.94 |   334 | 3,226,213 |    341 |
 | **NEGany~that_surprising**  | surprising  |    150,067.00 |    8.14 |    0.92 |  7,115.30 | 1,141 | 3,226,213 |  1,187 |
 | **NEGany~that_common**      | common      |    556,435.00 |    8.12 |    0.92 |  7,564.08 | 1,216 | 3,226,213 |  1,268 |
 | **NEGany~that_dissimilar**  | dissimilar  |      8,816.00 |    7.00 |    0.92 |  1,904.15 |   307 | 3,226,213 |    321 |
 | **NEGany~that_hard**        | hard        |    430,990.00 |    7.96 |    0.88 | 59,642.82 | 9,966 | 3,226,213 | 10,818 |
 | **NEGany~that_complicated** | complicated |    180,071.00 |    7.95 |    0.91 |  7,450.89 | 1,208 | 3,226,213 |  1,270 |
 | **NEGany~that_impressed**   | impressed   |    113,281.00 |    7.57 |    0.91 |  4,207.58 |   684 | 3,226,213 |    721 |
 | **NEGany~that_noticeable**  | noticeable  |     40,372.00 |    6.78 |    0.91 |  1,632.07 |   265 | 3,226,213 |    279 |
 | **NEGany~that_exciting**    | exciting    |    236,396.00 |    7.48 |    0.90 |  4,892.83 |   805 | 3,226,213 |    859 |


Top 10 `NEGmirror` "that_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                            | `adj`      |   `adj_total` |   `LRC` |   `dP1` |     `G2` |   `f` |    `f1` |   `f2` |
 |:---------------------------|:-----------|--------------:|--------:|--------:|---------:|------:|--------:|-------:|
 | **NEGmir~that_popular**    | popular    |      5,787.00 |    2.50 |    0.76 |   200.44 |    65 | 289,770 |     70 |
 | **NEGmir~that_interested** | interested |      9,258.00 |    2.42 |    0.76 |   190.06 |    62 | 289,770 |     67 |
 | **NEGmir~that_difficult**  | difficult  |     16,043.00 |    2.15 |    0.75 |   155.64 |    52 | 289,770 |     57 |
 | **NEGmir~that_hard**       | hard       |      7,311.00 |    2.31 |    0.74 |   168.31 |    57 | 289,770 |     63 |
 | **NEGmir~that_close**      | close      |     13,962.00 |    2.39 |    0.73 |   174.26 |    60 | 289,770 |     67 |
 | **NEGmir~that_simple**     | simple     |     25,382.00 |    4.34 |    0.73 | 1,370.94 |   473 | 289,770 |    529 |
 | **NEGmir~that_easy**       | easy       |     20,050.00 |    4.21 |    0.72 | 1,258.15 |   442 | 289,770 |    500 |
 | **NEGmir~that_great**      | great      |      5,819.00 |    3.52 |    0.67 |   728.46 |   282 | 289,770 |    340 |
 | **NEGmir~that_good**       | good       |     33,540.00 |    3.07 |    0.56 |   953.31 |   447 | 289,770 |    615 |
 | **NEGmir~that_big**        | big        |      7,859.00 |    3.06 |    0.70 |   309.58 |   113 | 289,770 |    131 |


#### 4. _before_

 No bigrams found in loaded `RBdirect` AM table.
 No bigrams found in loaded `NEGmirror` AM table.

#### 5. _any_


Top 10 `RBdirect` "any_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                          | `adj`     |   `adj_total` |   `LRC` |   `dP1` |     `G2` |   `f` |      `f1` |   `f2` |
 |:-------------------------|:----------|--------------:|--------:|--------:|---------:|------:|----------:|-------:|
 | **NEGany~any_happier**   | happier   |     19,501.00 |    4.65 |    0.53 | 3,488.76 |   830 | 3,226,213 |  1,472 |
 | **NEGany~any_simpler**   | simpler   |     26,094.00 |    3.09 |    0.30 |   671.74 |   228 | 3,226,213 |    672 |
 | **NEGany~any_clearer**   | clearer   |     13,369.00 |    3.21 |    0.30 | 1,051.22 |   357 | 3,226,213 |  1,053 |
 | **NEGany~any_different** | different |    909,864.00 |    2.98 |    0.24 | 2,270.24 |   910 | 3,226,213 |  3,313 |
 | **NEGany~any_younger**   | younger   |     29,805.00 |    2.37 |    0.19 |   544.17 |   256 | 3,226,213 |  1,121 |
 | **NEGany~any_worse**     | worse     |    214,166.00 |    2.47 |    0.16 | 3,165.88 | 1,693 | 3,226,213 |  8,487 |
 | **NEGany~any_bigger**    | bigger    |    130,470.00 |    2.27 |    0.17 |   688.06 |   357 | 3,226,213 |  1,735 |
 | **NEGany~any_harder**    | harder    |     99,332.00 |    1.98 |    0.15 |   395.22 |   227 | 3,226,213 |  1,221 |
 | **NEGany~any_safer**     | safer     |     26,779.00 |    1.73 |    0.12 |   346.68 |   235 | 3,226,213 |  1,471 |
 | **NEGany~any_easier**    | easier    |    237,680.00 |    1.95 |    0.11 | 2,164.75 | 1,607 | 3,226,213 | 10,860 |


Top 4 `NEGmirror` "any_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                       | `adj`   |   `adj_total` |   `LRC` |   `dP1` |     `G2` |   `f` |    `f1` |   `f2` |
 |:----------------------|:--------|--------------:|--------:|--------:|---------:|------:|--------:|-------:|
 | **NEGmir~any_better** | better  |     14,076.00 |    4.44 |    0.75 | 1,148.18 |   381 | 289,770 |    416 |
 | **NEGmir~any_easier** | easier  |      2,409.00 |    2.42 |    0.75 |   181.98 |    61 | 289,770 |     67 |
 | **NEGmir~any_worse**  | worse   |      8,490.00 |    2.87 |    0.72 |   248.63 |    88 | 289,770 |    100 |
 | **NEGmir~any_closer** | closer  |        986.00 |    2.21 |    0.68 |   149.62 |    56 | 289,770 |     66 |


#### 6. _ever_


Top 5 `RBdirect` "ever_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                         | `adj`   |   `adj_total` |   `LRC` |   `dP1` |     `G2` |   `f` |      `f1` |   `f2` |
 |:------------------------|:--------|--------------:|--------:|--------:|---------:|------:|----------:|-------:|
 | **NEGany~ever_simple**  | simple  |    427,167.00 |    5.54 |    0.77 | 1,142.04 |   212 | 3,226,213 |    262 |
 | **NEGany~ever_easy**    | easy    |    771,307.00 |    5.06 |    0.63 | 2,030.58 |   430 | 3,226,213 |    641 |
 | **NEGany~ever_good**    | good    |  2,037,285.00 |    3.76 |    0.40 | 1,178.00 |   332 | 3,226,213 |    756 |
 | **NEGany~ever_perfect** | perfect |    164,519.00 |    3.48 |    0.37 |   736.05 |   217 | 3,226,213 |    527 |
 | **NEGany~ever_able**    | able    |    428,268.00 |    1.81 |    0.13 |   363.95 |   234 | 3,226,213 |  1,398 |


Top 6 `NEGmirror` "ever_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                         | `adj`   |   `adj_total` |   `LRC` |   `dP1` |     `G2` |   `f` |    `f1` |   `f2` |
 |:------------------------|:--------|--------------:|--------:|--------:|---------:|------:|--------:|-------:|
 | **NEGmir~ever_easy**    | easy    |     20,050.00 |    3.21 |    0.83 | 1,311.83 |   367 | 289,770 |    368 |
 | **NEGmir~ever_perfect** | perfect |      3,708.00 |    2.38 |    0.83 |   735.10 |   207 | 289,770 |    208 |
 | **NEGmir~ever_good**    | good    |     33,540.00 |    4.72 |    0.82 | 1,034.95 |   298 | 289,770 |    302 |
 | **NEGmir~ever_wrong**   | wrong   |     20,866.00 |    2.56 |    0.82 |   349.21 |   102 | 289,770 |    104 |
 | **NEGmir~ever_free**    | free    |      5,043.00 |    1.97 |    0.81 |   231.61 |    69 | 289,770 |     71 |
 | **NEGmir~ever_able**    | able    |      6,448.00 |    3.66 |    0.79 |   437.65 |   136 | 289,770 |    143 |


#### 7. _yet_


Top 10 `RBdirect` "yet_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                          | `adj`     |   `adj_total` |   `LRC` |   `dP1` |      `G2` |    `f` |      `f1` |   `f2` |
 |:-------------------------|:----------|--------------:|--------:|--------:|----------:|-------:|----------:|-------:|
 | **NEGany~yet_clear**     | clear     |    491,108.00 |   10.26 |    0.95 | 67,924.56 | 10,553 | 3,226,213 | 10,693 |
 | **NEGany~yet_eligible**  | eligible  |     49,578.00 |    7.72 |    0.94 |  2,929.15 |    459 | 3,226,213 |    468 |
 | **NEGany~yet_official**  | official  |      9,778.00 |    7.33 |    0.94 |  2,236.98 |    353 | 3,226,213 |    362 |
 | **NEGany~yet_ready**     | ready     |    240,297.00 |    9.23 |    0.93 | 48,012.06 |  7,611 | 3,226,213 |  7,838 |
 | **NEGany~yet_certain**   | certain   |    104,544.00 |    8.12 |    0.93 |  5,491.41 |    874 | 3,226,213 |    903 |
 | **NEGany~yet_complete**  | complete  |    107,018.00 |    8.42 |    0.92 | 13,815.99 |  2,220 | 3,226,213 |  2,314 |
 | **NEGany~yet_sure**      | sure      |    844,981.00 |    8.37 |    0.92 | 12,379.79 |  1,990 | 3,226,213 |  2,075 |
 | **NEGany~yet_available** | available |    866,272.00 |    7.69 |    0.87 | 44,196.15 |  7,481 | 3,226,213 |  8,238 |
 | **NEGany~yet_right**     | right     |    204,572.00 |    6.50 |    0.92 |  1,254.20 |    202 | 3,226,213 |    211 |
 | **NEGany~yet_final**     | final     |      9,657.00 |    7.45 |    0.91 |  4,028.75 |    659 | 3,226,213 |    699 |

 No bigrams found in loaded `NEGmirror` AM table.

#### 8. _longer_


Top 5 `RBdirect` "longer_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                          | `adj`     |   `adj_total` |   `LRC` |   `dP1` |   `G2` |    `f` |       `f1` |   `f2` |
 |:-------------------------|:----------|--------------:|--------:|--------:|-------:|-------:|-----------:|-------:|
 | **COM~longer_lasting**   | lasting   |     24,344.00 |    1.44 |    0.04 | 244.09 |  3,860 | 83,102,035 |  3,866 |
 | **COM~longer_enough**    | enough    |    453,790.00 |    1.41 |    0.03 | 216.98 |  3,952 | 83,102,035 |  3,964 |
 | **COM~longer_able**      | able      |    428,268.00 |    2.28 |    0.03 | 623.67 | 11,677 | 83,102,035 | 11,716 |
 | **COM~longer_available** | available |    866,272.00 |    2.45 |    0.03 | 974.55 | 18,865 | 83,102,035 | 18,935 |
 | **COM~longer_necessary** | necessary |    187,396.00 |    1.27 |    0.03 | 220.07 |  5,365 | 83,102,035 |  5,399 |

 No bigrams found in loaded `NEGmirror` AM table.

#### 9. _immediately_


Top 5 `RBdirect` "immediately_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                                  | `adj`     |   `adj_total` |   `LRC` |   `dP1` |       `G2` |    `f` |      `f1` |   `f2` |
 |:---------------------------------|:----------|--------------:|--------:|--------:|-----------:|-------:|----------:|-------:|
 | **NEGany~immediately_possible**  | possible  |    364,265.00 |    7.68 |    0.90 |   6,269.26 |  1,027 | 3,226,213 |  1,091 |
 | **NEGany~immediately_clear**     | clear     |    491,108.00 |    8.32 |    0.90 | 153,302.22 | 25,276 | 3,226,213 | 27,066 |
 | **NEGany~immediately_available** | available |    866,272.00 |    5.77 |    0.66 | 102,962.94 | 21,297 | 3,226,213 | 30,725 |
 | **NEGany~immediately_able**      | able      |    428,268.00 |    4.87 |    0.58 |   2,851.84 |    639 | 3,226,213 |  1,036 |
 | **NEGany~immediately_obvious**   | obvious   |    193,498.00 |    4.59 |    0.49 |   9,043.23 |  2,258 | 3,226,213 |  4,305 |


Top 1 `NEGmirror` "immediately_*" bigrams (sorted by `dP1`; `LRC > 1`)


 |                                  | `adj`     |   `adj_total` |   `LRC` |   `dP1` |   `G2` |   `f` |    `f1` |   `f2` |
 |:---------------------------------|:----------|--------------:|--------:|--------:|-------:|------:|--------:|-------:|
 | **NEGmir~immediately_available** | available |     12,636.00 |    1.94 |    0.43 | 254.47 |   162 | 289,770 |    274 |



In [19]:
# %%

bigram_dfs['RBdirect'].filter(like='~before_', axis=0)

,l2,f,E11,am_log_likelihood,...,adv_total,adj_total,l1,first_char
key,,,,,,,,,
COM~before_dark,before_dark,72,69.82,2.12,...,681,72272,COMPLEMENT,b
NEGany~before_available,before_available,177,8.23,"1,020.91",...,681,666909,NEGATED,b


In [20]:
# %%

for key, info in samples_dict.items():
    if key in ('bigrams', 'adj'):
        key = f'ALL {key.replace("adj", "adjectives")}'
    formatted_iter = [
        f'_{a.replace("_", " ")}_' for a
        in (info['adj'] if isinstance(info, dict)
            else info)]
    print_iter(formatted_iter,
               header=f'1. _{key}_ ({len(formatted_iter)} unique)',
               bullet='1.', indent=3)


1. _necessarily_ (16 unique)
   1. _fun_
   1. _useful_
   1. _reliable_
   1. _proud_
   1. _easy_
   1. _right_
   1. _representative_
   1. _better_
   1. _surprising_
   1. _essential_
   1. _true_
   1. _wrong_
   1. _illegal_
   1. _indicative_
   1. _new_
   1. _bad_

1. _that_ (20 unique)
   1. _comfortable_
   1. _great_
   1. _thrilled_
   1. _clear_
   1. _fond_
   1. _popular_
   1. _dissimilar_
   1. _unusual_
   1. _surprising_
   1. _simple_
   1. _keen_
   1. _familiar_
   1. _complicated_
   1. _far-fetched_
   1. _uncommon_
   1. _easy_
   1. _big_
   1. _noticeable_
   1. _impressive_
   1. _hard_

1. _exactly_ (11 unique)
   1. _difficult_
   1. _cheap_
   1. _easy_
   1. _conducive_
   1. _surprising_
   1. _shocking_
   1. _pleasant_
   1. _famous_
   1. _sure_
   1. _clear_
   1. _new_

1. _any_ (16 unique)
   1. _brighter_
   1. _younger_
   1. _different_
   1. _cheaper_
   1. _better_
   1. _simpler_
   1. _smarter_
   1. _happier_
   1. _higher_
   1. _good_


1. _necessarily_ (16 unique)
   1. _fun_
   1. _useful_
   1. _reliable_
   1. _proud_
   1. _easy_
   1. _right_
   1. _representative_
   1. _better_
   1. _surprising_
   1. _essential_
   1. _true_
   1. _wrong_
   1. _illegal_
   1. _indicative_
   1. _new_
   1. _bad_

1. _that_ (20 unique)
   1. _comfortable_
   1. _great_
   1. _thrilled_
   1. _clear_
   1. _fond_
   1. _popular_
   1. _dissimilar_
   1. _unusual_
   1. _surprising_
   1. _simple_
   1. _keen_
   1. _familiar_
   1. _complicated_
   1. _far-fetched_
   1. _uncommon_
   1. _easy_
   1. _big_
   1. _noticeable_
   1. _impressive_
   1. _hard_

1. _exactly_ (11 unique)
   1. _difficult_
   1. _cheap_
   1. _easy_
   1. _conducive_
   1. _surprising_
   1. _shocking_
   1. _pleasant_
   1. _famous_
   1. _sure_
   1. _clear_
   1. _new_

1. _any_ (16 unique)
   1. _brighter_
   1. _younger_
   1. _different_
   1. _cheaper_
   1. _better_
   1. _simpler_
   1. _smarter_
   1. _happier_
   1. _higher_
   1. _good_
   1. _closer_
   1. _worse_
   1. _nicer_
   1. _clearer_
   1. _bigger_
   1. _easier_

1. _remotely_ (14 unique)
   1. _close_
   1. _believable_
   1. _interesting_
   1. _similar_
   1. _interested_
   1. _possible_
   1. _surprising_
   1. _accurate_
   1. _ready_
   1. _funny_
   1. _qualified_
   1. _true_
   1. _comparable_
   1. _new_

1. _ever_ (13 unique)
   1. _certain_
   1. _able_
   1. _easy_
   1. _perfect_
   1. _sick_
   1. _enough_
   1. _simple_
   1. _sure_
   1. _good_
   1. _boring_
   1. _closer_
   1. _black_
   1. _satisfied_

1. _yet_ (13 unique)
   1. _certain_
   1. _over_
   1. _final_
   1. _available_
   1. _mainstream_
   1. _convinced_
   1. _ready_
   1. _sure_
   1. _official_
   1. _online_
   1. _eligible_
   1. _complete_
   1. _clear_

1. _immediately_ (10 unique)
   1. _certain_
   1. _able_
   1. _available_
   1. _obvious_
   1. _intuitive_
   1. _forthcoming_
   1. _sure_
   1. _clear_
   1. _reachable_
   1. _possible_

1. _particularly_ (17 unique)
   1. _close_
   1. _athletic_
   1. _revolutionary_
   1. _remarkable_
   1. _comfortable_
   1. _surprising_
   1. _original_
   1. _forthcoming_
   1. _radical_
   1. _flashy_
   1. _religious_
   1. _novel_
   1. _wrong_
   1. _fast_
   1. _likeable_
   1. _likable_
   1. _new_

1. _inherently_ (9 unique)
   1. _problematic_
   1. _better_
   1. _negative_
   1. _improper_
   1. _good_
   1. _bad_
   1. _illegal_
   1. _wrong_
   1. _evil_

1. _terribly_ (17 unique)
   1. _different_
   1. _likely_
   1. _uncommon_
   1. _interesting_
   1. _reliable_
   1. _remarkable_
   1. _productive_
   1. _interested_
   1. _unusual_
   1. _surprising_
   1. _original_
   1. _special_
   1. _sure_
   1. _clear_
   1. _impressive_
   1. _popular_
   1. _new_

1. _ALL bigrams_ (156 unique)
   1. _particularly wrong_
   1. _inherently wrong_
   1. _that easy_
   1. _that popular_
   1. _exactly surprising_
   1. _any brighter_
   1. _terribly unusual_
   1. _ever sick_
   1. _exactly clear_
   1. _any good_
   1. _ever easy_
   1. _immediately sure_
   1. _remotely qualified_
   1. _inherently good_
   1. _particularly athletic_
   1. _terribly productive_
   1. _necessarily reliable_
   1. _any worse_
   1. _remotely new_
   1. _remotely close_
   1. _exactly conducive_
   1. _ever certain_
   1. _that familiar_
   1. _necessarily better_
   1. _exactly sure_
   1. _ever able_
   1. _inherently illegal_
   1. _particularly remarkable_
   1. _particularly close_
   1. _any clearer_
   1. _terribly remarkable_
   1. _that simple_
   1. _any nicer_
   1. _yet over_
   1. _yet sure_
   1. _that comfortable_
   1. _necessarily proud_
   1. _necessarily surprising_
   1. _immediately available_
   1. _exactly famous_
   1. _particularly comfortable_
   1. _exactly cheap_
   1. _exactly pleasant_
   1. _necessarily fun_
   1. _particularly radical_
   1. _any easier_
   1. _yet certain_
   1. _yet complete_
   1. _necessarily easy_
   1. _exactly difficult_
   1. _ever closer_
   1. _yet eligible_
   1. _exactly shocking_
   1. _terribly sure_
   1. _yet clear_
   1. _remotely funny_
   1. _remotely interesting_
   1. _that impressive_
   1. _particularly fast_
   1. _that hard_
   1. _necessarily useful_
   1. _ever satisfied_
   1. _necessarily wrong_
   1. _immediately clear_
   1. _any higher_
   1. _ever enough_
   1. _immediately possible_
   1. _that noticeable_
   1. _terribly surprising_
   1. _terribly likely_
   1. _remotely believable_
   1. _terribly clear_
   1. _inherently problematic_
   1. _terribly uncommon_
   1. _ever black_
   1. _necessarily true_
   1. _terribly interested_
   1. _that dissimilar_
   1. _necessarily indicative_
   1. _any simpler_
   1. _immediately certain_
   1. _inherently bad_
   1. _exactly easy_
   1. _any smarter_
   1. _yet available_
   1. _any happier_
   1. _any closer_
   1. _particularly original_
   1. _particularly religious_
   1. _yet online_
   1. _any bigger_
   1. _that thrilled_
   1. _yet convinced_
   1. _particularly likeable_
   1. _that great_
   1. _that unusual_
   1. _any cheaper_
   1. _yet official_
   1. _terribly original_
   1. _yet final_
   1. _ever sure_
   1. _remotely possible_
   1. _particularly surprising_
   1. _that far-fetched_
   1. _necessarily representative_
   1. _that surprising_
   1. _ever perfect_
   1. _remotely similar_
   1. _inherently improper_
   1. _that fond_
   1. _ever good_
   1. _remotely true_
   1. _terribly reliable_
   1. _remotely interested_
   1. _terribly new_
   1. _any younger_
   1. _terribly different_
   1. _that complicated_
   1. _immediately obvious_
   1. _particularly likable_
   1. _immediately intuitive_
   1. _necessarily new_
   1. _yet ready_
   1. _particularly flashy_
   1. _terribly special_
   1. _particularly forthcoming_
   1. _that big_
   1. _immediately reachable_
   1. _immediately able_
   1. _necessarily bad_
   1. _terribly popular_
   1. _inherently better_
   1. _any better_
   1. _remotely comparable_
   1. _ever boring_
   1. _particularly new_
   1. _terribly interesting_
   1. _exactly new_
   1. _that keen_
   1. _yet mainstream_
   1. _inherently negative_
   1. _that clear_
   1. _that uncommon_
   1. _necessarily right_
   1. _any different_
   1. _inherently evil_
   1. _particularly revolutionary_
   1. _ever simple_
   1. _terribly impressive_
   1. _necessarily essential_
   1. _particularly novel_
   1. _necessarily illegal_
   1. _immediately forthcoming_
   1. _remotely surprising_
   1. _remotely accurate_
   1. _remotely ready_

1. _ALL adjectives_ (106 unique)
   1. _close_
   1. _proud_
   1. _interested_
   1. _radical_
   1. _true_
   1. _different_
   1. _fond_
   1. _likely_
   1. _cheaper_
   1. _similar_
   1. _dissimilar_
   1. _interesting_
   1. _ready_
   1. _online_
   1. _difficult_
   1. _better_
   1. _representative_
   1. _sure_
   1. _eligible_
   1. _impressive_
   1. _possible_
   1. _evil_
   1. _right_
   1. _essential_
   1. _convinced_
   1. _flashy_
   1. _special_
   1. _simple_
   1. _religious_
   1. _keen_
   1. _complete_
   1. _likable_
   1. _complicated_
   1. _accurate_
   1. _far-fetched_
   1. _enough_
   1. _illegal_
   1. _likeable_
   1. _bigger_
   1. _easier_
   1. _younger_
   1. _available_
   1. _big_
   1. _negative_
   1. _simpler_
   1. _thrilled_
   1. _comparable_
   1. _brighter_
   1. _revolutionary_
   1. _productive_
   1. _comfortable_
   1. _great_
   1. _improper_
   1. _clear_
   1. _reachable_
   1. _intuitive_
   1. _certain_
   1. _obvious_
   1. _perfect_
   1. _unusual_
   1. _happier_
   1. _pleasant_
   1. _worse_
   1. _novel_
   1. _able_
   1. _familiar_
   1. _useful_
   1. _athletic_
   1. _sick_
   1. _funny_
   1. _qualified_
   1. _mainstream_
   1. _closer_
   1. _problematic_
   1. _fun_
   1. _uncommon_
   1. _final_
   1. _cheap_
   1. _easy_
   1. _forthcoming_
   1. _famous_
   1. _hard_
   1. _over_
   1. _conducive_
   1. _smarter_
   1. _original_
   1. _fast_
   1. _bad_
   1. _satisfied_
   1. _surprising_
   1. _good_
   1. _indicative_
   1. _nicer_
   1. _clearer_
   1. _higher_
   1. _new_
   1. _black_
   1. _shocking_
   1. _remarkable_
   1. _wrong_
   1. _reliable_
   1. _noticeable_
   1. _official_
   1. _boring_
   1. _popular_
   1. _believable_


In [21]:
# %%

NEG_bigrams_sample = pd.concat(
    (ad['both'] for ad in samples_dict.values() if isinstance(ad, dict))).sort_values('LRC', ascending=False)

In [26]:
top_NEGbigram_df_path =  OUT_DIR.joinpath(
    f'{TAG}-Top{K}_NEG-ADV_Top{bigram_k}-bigrams{bigram_floor}.{timestamp_today()}.csv')
print(top_NEGbigram_df_path)


/share/compling/projects/sanpi/results/top_AM/ALL/ALL-Top8/ALL-Top8_NEG-ADV_Top10-bigrams50.2024-07-28.csv


In [27]:
NEG_bigrams_sample.to_csv(top_NEGbigram_df_path)
nb_show_table(NEG_bigrams_sample.sort_values('LRC', ascending=False),
              outpath=top_NEGbigram_df_path.with_suffix('.md'))


|                                       |   `adj_total` |   `adv_total` |   `unexp_f` |   `P1` |   `odds_r_disc` |   `f2` |   `exp_f` |       `f1` |   `MI` |       `G2` | `adj`          |   `LRC` | `l1`       |   `dP1` | `l2`                       |    `t` |        `N` |    `f` | `adv`        |
|:--------------------------------------|--------------:|--------------:|------------:|-------:|----------------:|-------:|----------:|-----------:|-------:|-----------:|:---------------|--------:|:-----------|--------:|:---------------------------|-------:|-----------:|-------:|:-------------|
| **NEGany~yet_clear**                  |       349,214 |        95,763 |    9,949.56 |   0.99 |            3.51 | 10,476 |    456.44 |  3,173,660 |   1.36 |  64,409.97 | clear          |   10.77 | NEGATED    |    0.95 | yet_clear                  |  97.54 | 72,839,589 | 10,406 | yet          |
| **NEGany~yet_ready**                  |       141,590 |        95,763 |    7,169.91 |   0.99 |            3.2


|                                       |   `adj_total` |   `adv_total` |   `unexp_f` |   `P1` |   `odds_r_disc` |   `f2` |   `exp_f` |       `f1` |   `MI` |       `G2` | `adj`          |   `LRC` | `l1`       |   `dP1` | `l2`                       |    `t` |        `N` |    `f` | `adv`        |
|:--------------------------------------|--------------:|--------------:|------------:|-------:|----------------:|-------:|----------:|-----------:|-------:|-----------:|:---------------|--------:|:-----------|--------:|:---------------------------|-------:|-----------:|-------:|:-------------|
| **NEGany~yet_clear**                  |       349,214 |        95,763 |    9,949.56 |   0.99 |            3.51 | 10,476 |    456.44 |  3,173,660 |   1.36 |  64,409.97 | clear          |   10.77 | NEGATED    |    0.95 | yet_clear                  |  97.54 | 72,839,589 | 10,406 | yet          |
| **NEGany~yet_ready**                  |       141,590 |        95,763 |    7,169.91 |   0.99 |            3.22 |  7,599 |    331.09 |  3,173,660 |   1.36 |  45,985.07 | ready          |    9.93 | NEGATED    |    0.94 | yet_ready                  |  82.79 | 72,839,589 |  7,501 | yet          |
| **NEGany~necessarily_indicative**     |         8,148 |        48,947 |    1,328.00 |   0.99 |            3.42 |  1,400 |     61.00 |  3,173,660 |   1.36 |   8,577.54 | indicative     |    9.43 | NEGATED    |    0.95 | necessarily_indicative     |  35.63 | 72,839,589 |  1,389 | necessarily  |
| **NEGany~that_uncommon**              |        11,312 |       208,262 |      766.97 |   1.00 |            3.85 |    804 |     35.03 |  3,173,660 |   1.36 |   4,998.32 | uncommon       |    9.43 | NEGATED    |    0.95 | that_uncommon              |  27.08 | 72,839,589 |    802 | that         |
| **NEGany~exactly_easy**               |       579,827 |        58,643 |    1,019.25 |   0.99 |            3.49 |  1,073 |     46.75 |  3,173,660 |   1.36 |   6,596.91 | easy           |    9.32 | NEGATED    |    0.95 | exactly_easy               |  31.22 | 72,839,589 |  1,066 | exactly      |
| **NEGany~yet_complete**               |        86,361 |        95,763 |    2,077.80 |   0.98 |            3.14 |  2,208 |     96.20 |  3,173,660 |   1.35 |  13,277.09 | complete       |    9.20 | NEGATED    |    0.94 | yet_complete               |  44.56 | 72,839,589 |  2,174 | yet          |
| **NEGany~that_surprising**            |        70,540 |       208,262 |    1,083.20 |   0.99 |            3.37 |  1,143 |     49.80 |  3,173,660 |   1.36 |   6,986.81 | surprising     |    9.20 | NEGATED    |    0.95 | that_surprising            |  32.18 | 72,839,589 |  1,133 | that         |
| **NEGany~exactly_new**                |       253,862 |        58,643 |    1,310.52 |   0.99 |            3.24 |  1,388 |     60.48 |  3,173,660 |   1.36 |   8,410.32 | new            |    9.10 | NEGATED    |    0.94 | exactly_new                |  35.39 | 72,839,589 |  1,371 | exactly      |
| **NEGany~necessarily_easy**           |       579,827 |        48,947 |      869.05 |   0.99 |            3.37 |    917 |     39.95 |  3,173,660 |   1.36 |   5,605.64 | easy           |    9.01 | NEGATED    |    0.95 | necessarily_easy           |  28.82 | 72,839,589 |    909 | necessarily  |
| **NEGany~yet_final**                  |         5,860 |        95,763 |      611.98 |   1.00 |            3.60 |    643 |     28.02 |  3,173,660 |   1.36 |   3,972.92 | final          |    8.97 | NEGATED    |    0.95 | yet_final                  |  24.19 | 72,839,589 |    640 | yet          |
| **NEGany~exactly_cheap**              |        60,531 |        58,643 |      660.72 |   0.99 |            3.53 |    695 |     30.28 |  3,173,660 |   1.36 |   4,281.59 | cheap          |    8.96 | NEGATED    |    0.95 | exactly_cheap              |  25.13 | 72,839,589 |    691 | exactly      |
| **NEGany~yet_eligible**               |        23,252 |        95,763 |      428.48 |   1.00 |            4.29 |    448 |     19.52 |  3,173,660 |   1.36 |   2,807.56 | eligible       |    8.96 | NEGATED    |    0.96 | yet_eligible               |  20.24 | 72,839,589 |    448 | yet          |
| **NEGany~that_unusual**               |        71,234 |       208,262 |      933.95 |   0.99 |            3.27 |    988 |     43.05 |  3,173,660 |   1.36 |   6,003.05 | unusual        |    8.92 | NEGATED    |    0.95 | that_unusual               |  29.88 | 72,839,589 |    977 | that         |
| **NEGany~exactly_clear**              |       349,214 |        58,643 |    1,668.27 |   0.98 |            3.00 |  1,784 |     77.73 |  3,173,660 |   1.35 |  10,578.33 | clear          |    8.78 | NEGATED    |    0.94 | exactly_clear              |  39.92 | 72,839,589 |  1,746 | exactly      |
| **NEGany~exactly_surprising**         |        70,540 |        58,643 |      420.79 |   1.00 |            3.81 |    441 |     19.21 |  3,173,660 |   1.36 |   2,743.34 | surprising     |    8.71 | NEGATED    |    0.95 | exactly_surprising         |  20.06 | 72,839,589 |    440 | exactly      |
| **NEGany~that_complicated**           |       159,822 |       208,262 |    1,153.41 |   0.98 |            3.05 |  1,230 |     53.59 |  3,173,660 |   1.35 |   7,337.84 | complicated    |    8.68 | NEGATED    |    0.94 | that_complicated           |  33.20 | 72,839,589 |  1,207 | that         |
| **NEGany~terribly_surprising**        |        70,540 |        58,964 |      907.00 |   0.98 |            3.13 |    964 |     42.00 |  3,173,660 |   1.35 |   5,794.10 | surprising     |    8.66 | NEGATED    |    0.94 | terribly_surprising        |  29.44 | 72,839,589 |    949 | terribly     |
| **NEGany~yet_official**               |         6,853 |        95,763 |      336.66 |   1.00 |            4.19 |    352 |     15.34 |  3,173,660 |   1.36 |   2,205.93 | official       |    8.61 | NEGATED    |    0.96 | yet_official               |  17.94 | 72,839,589 |    352 | yet          |
| **NEGany~that_hard**                  |       348,463 |       208,262 |    9,495.74 |   0.96 |            2.70 | 10,380 |    452.26 |  3,173,660 |   1.34 |  58,817.24 | hard           |    8.59 | NEGATED    |    0.91 | that_hard                  |  95.21 | 72,839,589 |  9,948 | that         |
| **NEGany~that_familiar**              |       156,296 |       208,262 |    1,075.63 |   0.97 |            2.91 |  1,156 |     50.37 |  3,173,660 |   1.35 |   6,781.11 | familiar       |    8.35 | NEGATED    |    0.93 | that_familiar              |  32.05 | 72,839,589 |  1,126 | that         |
| **NEGany~necessarily_representative** |        18,355 |        48,947 |      465.56 |   0.99 |            3.29 |    492 |     21.44 |  3,173,660 |   1.36 |   2,996.58 | representative |    8.34 | NEGATED    |    0.95 | necessarily_representative |  21.10 | 72,839,589 |    487 | necessarily  |
| **NEGany~necessarily_surprising**     |        70,540 |        48,947 |      325.14 |   1.00 |            3.70 |    341 |     14.86 |  3,173,660 |   1.36 |   2,117.16 | surprising     |    8.33 | NEGATED    |    0.95 | necessarily_surprising     |  17.63 | 72,839,589 |    340 | necessarily  |
| **NEGany~immediately_clear**          |       349,214 |        96,973 |   23,281.51 |   0.94 |            2.52 | 26,038 |  1,134.49 |  3,173,660 |   1.33 | 141,186.69 | clear          |    8.16 | NEGATED    |    0.89 | immediately_clear          | 149.00 | 72,839,589 | 24,416 | immediately  |
| **NEGany~necessarily_new**            |       253,862 |        48,947 |      460.56 |   0.98 |            3.00 |    492 |     21.44 |  3,173,660 |   1.35 |   2,923.82 | new            |    7.94 | NEGATED    |    0.94 | necessarily_new            |  20.98 | 72,839,589 |    482 | necessarily  |
| **NEGany~that_dissimilar**            |         4,605 |       208,262 |      290.62 |   0.99 |            3.28 |    307 |     13.38 |  3,173,660 |   1.36 |   1,871.65 | dissimilar     |    7.86 | NEGATED    |    0.95 | that_dissimilar            |  16.67 | 72,839,589 |    304 | that         |
| **NEGany~exactly_conducive**          |         9,110 |        58,643 |      198.94 |   1.00 |            3.96 |    208 |      9.06 |  3,173,660 |   1.36 |   1,303.50 | conducive      |    7.82 | NEGATED    |    0.96 | exactly_conducive          |  13.79 | 72,839,589 |    208 | exactly      |
| **NEGany~that_noticeable**            |        31,467 |       208,262 |      252.37 |   0.99 |            3.22 |    267 |     11.63 |  3,173,660 |   1.36 |   1,621.81 | noticeable     |    7.65 | NEGATED    |    0.95 | that_noticeable            |  15.53 | 72,839,589 |    264 | that         |
| **NEGany~immediately_possible**       |       245,272 |        96,973 |      954.08 |   0.95 |            2.61 |  1,054 |     45.92 |  3,173,660 |   1.34 |   5,845.77 | possible       |    7.62 | NEGATED    |    0.91 | immediately_possible       |  30.17 | 72,839,589 |  1,000 | immediately  |
| **NEGany~yet_convinced**              |        12,132 |        95,763 |      161.64 |   1.00 |            3.87 |    169 |      7.36 |  3,173,660 |   1.36 |   1,059.09 | convinced      |    7.50 | NEGATED    |    0.96 | yet_convinced              |  12.43 | 72,839,589 |    169 | yet          |
| **NEGany~exactly_shocking**           |        35,115 |        58,643 |      144.42 |   1.00 |            3.82 |    151 |      6.58 |  3,173,660 |   1.36 |     946.29 | shocking       |    7.33 | NEGATED    |    0.96 | exactly_shocking           |  11.75 | 72,839,589 |    151 | exactly      |
| **NEGany~exactly_pleasant**           |        52,223 |        58,643 |      135.81 |   1.00 |            3.80 |    142 |      6.19 |  3,173,660 |   1.36 |     889.88 | pleasant       |    7.24 | NEGATED    |    0.96 | exactly_pleasant           |  11.40 | 72,839,589 |    142 | exactly      |
| **NEGany~yet_over**                   |         3,983 |        95,763 |      154.90 |   0.99 |            3.38 |    163 |      7.10 |  3,173,660 |   1.36 |   1,003.13 | over           |    7.21 | NEGATED    |    0.95 | yet_over                   |  12.17 | 72,839,589 |    162 | yet          |
| **NEGany~exactly_famous**             |       223,813 |        58,643 |      124.34 |   1.00 |            3.76 |    130 |      5.66 |  3,173,660 |   1.36 |     814.68 | famous         |    7.10 | NEGATED    |    0.96 | exactly_famous             |  10.90 | 72,839,589 |    130 | exactly      |
| **NEGany~exactly_difficult**          |       732,106 |        58,643 |      120.51 |   1.00 |            3.74 |    126 |      5.49 |  3,173,660 |   1.36 |     789.62 | difficult      |    7.05 | NEGATED    |    0.96 | exactly_difficult          |  10.74 | 72,839,589 |    126 | exactly      |
| **NEGany~necessarily_useful**         |       227,709 |        48,947 |       99.47 |   1.00 |            3.66 |    104 |      4.53 |  3,173,660 |   1.36 |     651.75 | useful         |    6.75 | NEGATED    |    0.96 | necessarily_useful         |   9.75 | 72,839,589 |    104 | necessarily  |
| **NEGany~yet_online**                 |        15,650 |        95,763 |       93.73 |   1.00 |            3.64 |     98 |      4.27 |  3,173,660 |   1.36 |     614.14 | online         |    6.66 | NEGATED    |    0.96 | yet_online                 |   9.47 | 72,839,589 |     98 | yet          |
| **NEGany~necessarily_fun**            |       190,026 |        48,947 |       91.82 |   1.00 |            3.63 |     96 |      4.18 |  3,173,660 |   1.36 |     601.61 | fun            |    6.62 | NEGATED    |    0.96 | necessarily_fun            |   9.37 | 72,839,589 |     96 | necessarily  |
| **NEGany~necessarily_essential**      |        69,845 |        48,947 |       88.95 |   1.00 |            3.61 |     93 |      4.05 |  3,173,660 |   1.36 |     582.81 | essential      |    6.57 | NEGATED    |    0.96 | necessarily_essential      |   9.22 | 72,839,589 |     93 | necessarily  |
| **NEGmir~ever_easy**                  |        18,610 |         5,060 |      304.75 |   1.00 |            3.08 |    369 |     63.25 |    291,732 |   0.76 |   1,285.01 | easy           |    6.44 | NEGMIR     |    0.83 | ever_easy                  |  15.89 |  1,701,929 |    368 | ever         |
| **NEGany~terribly_original**          |        37,594 |        58,964 |      189.76 |   0.94 |            2.51 |    212 |      9.24 |  3,173,660 |   1.33 |   1,150.48 | original       |    6.41 | NEGATED    |    0.90 | terribly_original          |  13.45 | 72,839,589 |    199 | terribly     |
| **NEGany~necessarily_reliable**       |        90,598 |        48,947 |       77.47 |   1.00 |            3.55 |     81 |      3.53 |  3,173,660 |   1.36 |     507.61 | reliable       |    6.35 | NEGATED    |    0.96 | necessarily_reliable       |   8.61 | 72,839,589 |     81 | necessarily  |
| **NEGany~any_happier**                |        16,606 |        34,382 |      786.04 |   0.86 |            2.13 |    963 |     41.96 |  3,173,660 |   1.30 |   4,420.49 | happier        |    6.34 | NEGATED    |    0.82 | any_happier                |  27.32 | 72,839,589 |    828 | any          |
| **NEGany~terribly_different**         |       825,838 |        58,964 |      348.18 |   0.89 |            2.27 |    409 |     17.82 |  3,173,660 |   1.31 |   2,022.47 | different      |    6.33 | NEGATED    |    0.85 | terribly_different         |  18.20 | 72,839,589 |    366 | terribly     |
| **NEGany~terribly_uncommon**          |        11,312 |        58,964 |       98.43 |   0.98 |            2.96 |    105 |      4.57 |  3,173,660 |   1.35 |     625.85 | uncommon       |    6.33 | NEGATED    |    0.94 | terribly_uncommon          |   9.70 | 72,839,589 |    103 | terribly     |
| **NEGany~terribly_likely**            |       890,421 |        58,964 |      103.16 |   0.97 |            2.83 |    111 |      4.84 |  3,173,660 |   1.35 |     649.50 | likely         |    6.26 | NEGATED    |    0.93 | terribly_likely            |   9.93 | 72,839,589 |    108 | terribly     |
| **NEGany~terribly_unusual**           |        71,234 |        58,964 |      139.25 |   0.94 |            2.53 |    155 |      6.75 |  3,173,660 |   1.33 |     847.05 | unusual        |    6.17 | NEGATED    |    0.90 | terribly_unusual           |  11.52 | 72,839,589 |    146 | terribly     |
| **NEGany~necessarily_proud**          |       207,536 |        48,947 |       67.91 |   1.00 |            3.50 |     71 |      3.09 |  3,173,660 |   1.36 |     444.94 | proud          |    6.14 | NEGATED    |    0.96 | necessarily_proud          |   8.06 | 72,839,589 |     71 | necessarily  |
| **NEGany~particularly_religious**     |        28,028 |       513,668 |      460.38 |   0.86 |            2.12 |    565 |     24.62 |  3,173,660 |   1.29 |   2,585.71 | religious      |    6.12 | NEGATED    |    0.81 | particularly_religious     |  20.90 | 72,839,589 |    485 | particularly |
| **NEGany~yet_mainstream**             |        17,792 |        95,763 |       66.95 |   1.00 |            3.49 |     70 |      3.05 |  3,173,660 |   1.36 |     438.67 | mainstream     |    6.11 | NEGATED    |    0.96 | yet_mainstream             |   8.00 | 72,839,589 |     70 | yet          |
| **NEGany~particularly_new**           |       253,862 |       513,668 |      707.79 |   0.83 |            2.03 |    900 |     39.21 |  3,173,660 |   1.28 |   3,874.46 | new            |    6.04 | NEGATED    |    0.79 | particularly_new           |  25.90 | 72,839,589 |    747 | particularly |
| **NEGany~terribly_impressive**        |       178,051 |        58,964 |      186.50 |   0.90 |            2.28 |    218 |      9.50 |  3,173,660 |   1.31 |   1,087.64 | impressive     |    5.97 | NEGATED    |    0.86 | terribly_impressive        |  13.32 | 72,839,589 |    196 | terribly     |
| **NEGmir~particularly_new**           |        12,836 |        13,003 |      333.72 |   0.99 |            2.48 |    410 |     70.28 |    291,732 |   0.76 |   1,365.17 | new            |    5.92 | NEGMIR     |    0.81 | particularly_new           |  16.60 |  1,701,929 |    404 | particularly |
| **NEGany~particularly_original**      |        37,594 |       513,668 |      341.44 |   0.85 |            2.08 |    426 |     18.56 |  3,173,660 |   1.29 |   1,894.60 | original       |    5.86 | NEGATED    |    0.80 | particularly_original      |  18.00 | 72,839,589 |    360 | particularly |
| **NEGany~that_far-fetched**           |         5,185 |       208,262 |       56.43 |   1.00 |            3.42 |     59 |      2.57 |  3,173,660 |   1.36 |     369.74 | far-fetched    |    5.83 | NEGATED    |    0.96 | that_far-fetched           |   7.35 | 72,839,589 |     59 | that         |
| **NEGany~that_thrilled**              |        24,182 |       208,262 |       56.43 |   1.00 |            3.42 |     59 |      2.57 |  3,173,660 |   1.36 |     369.74 | thrilled       |    5.83 | NEGATED    |    0.96 | that_thrilled              |   7.35 | 72,839,589 |     59 | that         |
| **NEGmir~ever_simple**                |        25,408 |         5,060 |      170.69 |   1.00 |            3.30 |    206 |     35.31 |    291,732 |   0.77 |     726.76 | simple         |    5.82 | NEGMIR     |    0.83 | ever_simple                |  11.89 |  1,701,929 |    206 | ever         |
| **NEGany~immediately_available**      |       666,909 |        96,973 |   19,799.16 |   0.72 |            1.75 | 29,351 |  1,278.84 |  3,173,660 |   1.22 |  98,046.67 | available      |    5.70 | NEGATED    |    0.67 | immediately_available      | 136.37 | 72,839,589 | 21,078 | immediately  |
| **NEGmir~ever_good**                  |        31,585 |         5,060 |      247.06 |   0.99 |            2.51 |    303 |     51.94 |    291,732 |   0.76 |   1,013.87 | good           |    5.68 | NEGMIR     |    0.82 | ever_good                  |  14.29 |  1,701,929 |    299 | ever         |
| **NEGany~any_younger**                |        26,216 |        34,382 |      241.62 |   0.83 |            2.03 |    307 |     13.38 |  3,173,660 |   1.28 |   1,323.36 | younger        |    5.57 | NEGATED    |    0.79 | any_younger                |  15.13 | 72,839,589 |    255 | any          |
| **NEGmir~ever_perfect**               |         3,134 |         5,060 |      170.52 |   1.00 |            2.82 |    207 |     35.48 |    291,732 |   0.76 |     714.47 | perfect        |    5.57 | NEGMIR     |    0.82 | ever_perfect               |  11.88 |  1,701,929 |    206 | ever         |
| **NEGany~immediately_reachable**      |         2,672 |        96,973 |      103.77 |   0.91 |            2.32 |    120 |      5.23 |  3,173,660 |   1.32 |     610.53 | reachable      |    5.56 | NEGATED    |    0.86 | immediately_reachable      |   9.94 | 72,839,589 |    109 | immediately  |
| **NEGany~ever_simple**                |       396,749 |       114,075 |      200.11 |   0.84 |            2.07 |    250 |     10.89 |  3,173,660 |   1.29 |   1,109.28 | simple         |    5.56 | NEGATED    |    0.80 | ever_simple                |  13.78 | 72,839,589 |    211 | ever         |
| **NEGany~particularly_likable**       |         8,160 |       513,668 |      113.21 |   0.89 |            2.26 |    133 |      5.79 |  3,173,660 |   1.31 |     657.49 | likable        |    5.53 | NEGATED    |    0.85 | particularly_likable       |  10.38 | 72,839,589 |    119 | particularly |
| **NEGany~ever_easy**                  |       579,827 |       114,075 |      404.82 |   0.77 |            1.87 |    555 |     24.18 |  3,173,660 |   1.25 |   2,105.13 | easy           |    5.41 | NEGATED    |    0.73 | ever_easy                  |  19.54 | 72,839,589 |    429 | ever         |
| **NEGany~immediately_sure**           |       262,825 |        96,973 |      131.03 |   0.86 |            2.13 |    160 |      6.97 |  3,173,660 |   1.30 |     738.65 | sure           |    5.40 | NEGATED    |    0.82 | immediately_sure           |  11.15 | 72,839,589 |    138 | immediately  |
| **NEGany~terribly_productive**        |       102,361 |        58,964 |       61.08 |   0.96 |            2.61 |     67 |      2.92 |  3,173,660 |   1.34 |     376.84 | productive     |    5.39 | NEGATED    |    0.91 | terribly_productive        |   7.64 | 72,839,589 |     64 | terribly     |
| **NEGany~inherently_wrong**           |       149,064 |        47,803 |    1,538.13 |   0.71 |            1.73 |  2,315 |    100.87 |  3,173,660 |   1.21 |   7,535.97 | wrong          |    5.36 | NEGATED    |    0.66 | inherently_wrong           |  37.99 | 72,839,589 |  1,639 | inherently   |
| **NEGany~particularly_likeable**      |         5,902 |       513,668 |      100.77 |   0.88 |            2.21 |    120 |      5.23 |  3,173,660 |   1.31 |     579.07 | likeable       |    5.34 | NEGATED    |    0.84 | particularly_likeable      |   9.79 | 72,839,589 |    106 | particularly |
| **NEGmir~exactly_sure**               |         6,761 |         1,041 |      122.63 |   1.00 |            3.16 |    148 |     25.37 |    291,732 |   0.77 |     522.11 | sure           |    5.31 | NEGMIR     |    0.83 | exactly_sure               |  10.08 |  1,701,929 |    148 | exactly      |
| **NEGmir~ever_enough**                |         2,596 |         5,060 |      121.80 |   1.00 |            3.15 |    147 |     25.20 |    291,732 |   0.77 |     518.58 | enough         |    5.30 | NEGMIR     |    0.83 | ever_enough                |  10.05 |  1,701,929 |    147 | ever         |
| **NEGmir~ever_certain**               |         1,800 |         5,060 |      118.49 |   1.00 |            3.14 |    143 |     24.51 |    291,732 |   0.77 |     504.47 | certain        |    5.26 | NEGMIR     |    0.83 | ever_certain               |   9.91 |  1,701,929 |    143 | ever         |
| **NEGany~terribly_sure**              |       262,825 |        58,964 |       65.69 |   0.91 |            2.31 |     76 |      3.31 |  3,173,660 |   1.32 |     386.31 | sure           |    5.08 | NEGATED    |    0.86 | terribly_sure              |   7.91 | 72,839,589 |     69 | terribly     |
| **NEGany~any_easier**                 |       209,940 |        34,382 |    1,489.21 |   0.66 |            1.64 |  2,405 |    104.79 |  3,173,660 |   1.18 |   6,987.78 | easier         |    5.08 | NEGATED    |    0.62 | any_easier                 |  37.30 | 72,839,589 |  1,594 | any          |
| **NEGany~remotely_surprising**        |        70,540 |        16,426 |       71.34 |   0.89 |            2.24 |     84 |      3.66 |  3,173,660 |   1.31 |     413.61 | surprising     |    5.07 | NEGATED    |    0.85 | remotely_surprising        |   8.24 | 72,839,589 |     75 | remotely     |
| **NEGmir~particularly_surprising**    |         2,662 |        13,003 |      137.20 |   0.99 |            2.51 |    168 |     28.80 |    291,732 |   0.76 |     564.67 | surprising     |    5.06 | NEGMIR     |    0.82 | particularly_surprising    |  10.65 |  1,701,929 |    166 | particularly |
| **NEGmir~particularly_wrong**         |        20,880 |        13,003 |      174.80 |   0.98 |            2.27 |    217 |     37.20 |    291,732 |   0.76 |     702.22 | wrong          |    5.05 | NEGMIR     |    0.81 | particularly_wrong         |  12.01 |  1,701,929 |    212 | particularly |
| **NEGmir~necessarily_wrong**          |        20,880 |         1,107 |      173.97 |   0.98 |            2.27 |    216 |     37.03 |    291,732 |   0.76 |     698.74 | wrong          |    5.04 | NEGMIR     |    0.81 | necessarily_wrong          |  11.98 |  1,701,929 |    211 | necessarily  |
| **NEGany~any_simpler**                |        23,480 |        34,382 |      212.93 |   0.75 |            1.82 |    300 |     13.07 |  3,173,660 |   1.24 |   1,087.71 | simpler        |    5.00 | NEGATED    |    0.71 | any_simpler                |  14.16 | 72,839,589 |    226 | any          |
| **NEGany~terribly_reliable**          |        90,598 |        58,964 |       48.65 |   0.94 |            2.51 |     54 |      2.35 |  3,173,660 |   1.34 |     296.70 | reliable       |    4.99 | NEGATED    |    0.90 | terribly_reliable          |   6.81 | 72,839,589 |     51 | terribly     |
| **NEGany~particularly_forthcoming**   |         7,473 |       513,668 |       68.38 |   0.87 |            2.14 |     83 |      3.62 |  3,173,660 |   1.30 |     387.25 | forthcoming    |    4.85 | NEGATED    |    0.82 | particularly_forthcoming   |   8.06 | 72,839,589 |     72 | particularly |
| **NEGany~ever_certain**               |        74,952 |       114,075 |      138.59 |   0.76 |            1.84 |    193 |      8.41 |  3,173,660 |   1.24 |     713.34 | certain        |    4.80 | NEGATED    |    0.72 | ever_certain               |  11.43 | 72,839,589 |    147 | ever         |
| **NEGany~particularly_athletic**      |        17,142 |       513,668 |      102.07 |   0.79 |            1.92 |    136 |      5.93 |  3,173,660 |   1.26 |     541.01 | athletic       |    4.77 | NEGATED    |    0.75 | particularly_athletic      |   9.82 | 72,839,589 |    108 | particularly |
| **NEGany~any_nicer**                  |         9,955 |        34,382 |       90.82 |   0.81 |            1.95 |    119 |      5.18 |  3,173,660 |   1.27 |     486.82 | nicer          |    4.75 | NEGATED    |    0.76 | any_nicer                  |   9.27 | 72,839,589 |     96 | any          |
| **NEGany~particularly_surprising**    |        70,540 |       513,668 |      993.06 |   0.61 |            1.54 |  1,743 |     75.94 |  3,173,660 |   1.15 |   4,433.52 | surprising     |    4.71 | NEGATED    |    0.57 | particularly_surprising    |  30.37 | 72,839,589 |  1,069 | particularly |
| **NEGany~immediately_able**           |       223,196 |        96,973 |      582.91 |   0.63 |            1.58 |    989 |     43.09 |  3,173,660 |   1.16 |   2,655.18 | able           |    4.70 | NEGATED    |    0.59 | immediately_able           |  23.30 | 72,839,589 |    626 | immediately  |
| **NEGany~inherently_bad**             |       429,537 |        47,803 |      737.05 |   0.61 |            1.53 |  1,307 |     56.95 |  3,173,660 |   1.14 |   3,270.68 | bad            |    4.62 | NEGATED    |    0.56 | inherently_bad             |  26.16 | 72,839,589 |    794 | inherently   |
| **NEGany~remotely_true**              |       231,639 |        16,426 |      233.88 |   0.68 |            1.66 |    370 |     16.12 |  3,173,660 |   1.19 |   1,111.13 | true           |    4.61 | NEGATED    |    0.63 | remotely_true              |  14.79 | 72,839,589 |    250 | remotely     |
| **NEGany~immediately_obvious**        |       165,439 |        96,973 |    2,064.20 |   0.56 |            1.45 |  3,989 |    173.80 |  3,173,660 |   1.11 |   8,712.07 | obvious        |    4.55 | NEGATED    |    0.52 | immediately_obvious        |  43.63 | 72,839,589 |  2,238 | immediately  |
| **NEGany~particularly_revolutionary** |        10,338 |       513,668 |       72.77 |   0.79 |            1.92 |     97 |      4.23 |  3,173,660 |   1.26 |     385.60 | revolutionary  |    4.49 | NEGATED    |    0.75 | particularly_revolutionary |   8.29 | 72,839,589 |     77 | particularly |
| **NEGany~ever_boring**                |        45,891 |       114,075 |       68.08 |   0.80 |            1.93 |     90 |      3.92 |  3,173,660 |   1.26 |     362.74 | boring         |    4.46 | NEGATED    |    0.76 | ever_boring                |   8.02 | 72,839,589 |     72 | ever         |
| **NEGmir~any_better**                 |        14,013 |         1,197 |      308.18 |   0.91 |            1.67 |    419 |     71.82 |    291,732 |   0.72 |   1,096.01 | better         |    4.38 | NEGMIR     |    0.74 | any_better                 |  15.81 |  1,701,929 |    380 | any          |
| **NEGmir~that_simple**                |        25,408 |         5,494 |      383.32 |   0.90 |            1.62 |    529 |     90.68 |    291,732 |   0.72 |   1,340.19 | simple         |    4.36 | NEGMIR     |    0.72 | that_simple                |  17.61 |  1,701,929 |    474 | that         |
| **NEGany~any_clearer**                |        11,680 |        34,382 |      328.51 |   0.58 |            1.49 |    608 |     26.49 |  3,173,660 |   1.13 |   1,421.60 | clearer        |    4.26 | NEGATED    |    0.54 | any_clearer                |  17.44 | 72,839,589 |    355 | any          |
| **NEGmir~particularly_remarkable**    |         3,238 |        13,003 |       88.97 |   0.97 |            2.18 |    111 |     19.03 |    291,732 |   0.75 |     354.53 | remarkable     |    4.24 | NEGMIR     |    0.80 | particularly_remarkable    |   8.56 |  1,701,929 |    108 | particularly |
| **NEGmir~that_easy**                  |        18,610 |         5,494 |      362.92 |   0.89 |            1.57 |    508 |     87.08 |    291,732 |   0.71 |   1,248.84 | easy           |    4.23 | NEGMIR     |    0.71 | that_easy                  |  17.11 |  1,701,929 |    450 | that         |
| **NEGany~immediately_certain**        |        74,952 |        96,973 |       65.95 |   0.75 |            1.82 |     93 |      4.05 |  3,173,660 |   1.24 |     336.68 | certain        |    4.19 | NEGATED    |    0.71 | immediately_certain        |   7.88 | 72,839,589 |     70 | immediately  |
| **NEGmir~ever_able**                  |         3,704 |         5,060 |      111.49 |   0.95 |            1.94 |    143 |     24.51 |    291,732 |   0.74 |     426.52 | able           |    4.17 | NEGMIR     |    0.78 | ever_able                  |   9.56 |  1,701,929 |    136 | ever         |
| **NEGany~remotely_ready**             |       141,590 |        16,426 |       54.78 |   0.78 |            1.89 |     74 |      3.22 |  3,173,660 |   1.26 |     287.63 | ready          |    4.16 | NEGATED    |    0.74 | remotely_ready             |   7.19 | 72,839,589 |     58 | remotely     |
| **NEGany~remotely_funny**             |       122,927 |        16,426 |      127.76 |   0.65 |            1.60 |    212 |      9.24 |  3,173,660 |   1.17 |     589.74 | funny          |    4.15 | NEGATED    |    0.60 | remotely_funny             |  10.92 | 72,839,589 |    137 | remotely     |
| **NEGmir~inherently_wrong**           |        20,880 |         5,133 |    1,200.00 |   0.83 |            1.37 |  1,826 |    313.00 |    291,732 |   0.68 |   3,787.53 | wrong          |    4.08 | NEGMIR     |    0.66 | inherently_wrong           |  30.85 |  1,701,929 |  1,513 | inherently   |
| **NEGany~particularly_flashy**        |         4,494 |       513,668 |       53.78 |   0.77 |            1.86 |     74 |      3.22 |  3,173,660 |   1.25 |     278.96 | flashy         |    4.08 | NEGATED    |    0.73 | particularly_flashy        |   7.12 | 72,839,589 |     57 | particularly |
| **NEGmir~terribly_surprising**        |         2,662 |         4,610 |       55.52 |   1.00 |            2.81 |     67 |     11.48 |    291,732 |   0.77 |     236.35 | surprising     |    4.07 | NEGMIR     |    0.83 | terribly_surprising        |   6.78 |  1,701,929 |     67 | terribly     |
| **NEGany~any_cheaper**                |        46,055 |        34,382 |      120.02 |   0.63 |            1.56 |    206 |      8.98 |  3,173,660 |   1.16 |     542.97 | cheaper        |    4.01 | NEGATED    |    0.58 | any_cheaper                |  10.57 | 72,839,589 |    129 | any          |
| **NEGmir~particularly_close**         |        13,874 |        13,003 |      111.15 |   0.94 |            1.84 |    145 |     24.85 |    291,732 |   0.74 |     415.70 | close          |    4.01 | NEGMIR     |    0.77 | particularly_close         |   9.53 |  1,701,929 |    136 | particularly |
| **NEGany~any_smarter**                |         8,501 |        34,382 |       83.25 |   0.67 |            1.65 |    132 |      5.75 |  3,173,660 |   1.19 |     394.95 | smarter        |    4.00 | NEGATED    |    0.63 | any_smarter                |   8.82 | 72,839,589 |     89 | any          |
| **NEGany~any_worse**                  |       179,012 |        34,382 |    1,525.97 |   0.46 |            1.27 |  3,673 |    160.03 |  3,173,660 |   1.02 |   5,676.37 | worse          |    3.94 | NEGATED    |    0.42 | any_worse                  |  37.16 | 72,839,589 |  1,686 | any          |
| **NEGany~any_brighter**               |         9,280 |        34,382 |       59.17 |   0.72 |            1.74 |     88 |      3.83 |  3,173,660 |   1.22 |     292.00 | brighter       |    3.91 | NEGATED    |    0.67 | any_brighter               |   7.45 | 72,839,589 |     63 | any          |
| **NEGany~immediately_forthcoming**    |         7,473 |        96,973 |      123.28 |   0.60 |            1.51 |    223 |      9.72 |  3,173,660 |   1.14 |     540.70 | forthcoming    |    3.89 | NEGATED    |    0.55 | immediately_forthcoming    |  10.69 | 72,839,589 |    133 | immediately  |
| **NEGany~ever_good**                  |     1,681,795 |       114,075 |      302.24 |   0.50 |            1.34 |    660 |     28.76 |  3,173,660 |   1.06 |   1,188.69 | good           |    3.81 | NEGATED    |    0.46 | ever_good                  |  16.61 | 72,839,589 |    331 | ever         |
| **NEGmir~ever_boring**                |         1,961 |         5,060 |       47.23 |   1.00 |            2.75 |     57 |      9.77 |    291,732 |   0.77 |     201.07 | boring         |    3.80 | NEGMIR     |    0.83 | ever_boring                |   6.26 |  1,701,929 |     57 | ever         |
| **NEGmir~ever_black**                 |         1,412 |         5,060 |       46.40 |   1.00 |            2.74 |     56 |      9.60 |    291,732 |   0.77 |     197.54 | black          |    3.77 | NEGMIR     |    0.83 | ever_black                 |   6.20 |  1,701,929 |     56 | ever         |
| **NEGmir~particularly_novel**         |           320 |        13,003 |       44.74 |   1.00 |            2.72 |     54 |      9.26 |    291,732 |   0.77 |     190.49 | novel          |    3.71 | NEGMIR     |    0.83 | particularly_novel         |   6.09 |  1,701,929 |     54 | particularly |
| **NEGmir~terribly_new**               |        12,836 |         4,610 |       56.83 |   0.97 |            2.13 |     71 |     12.17 |    291,732 |   0.75 |     225.93 | new            |    3.66 | NEGMIR     |    0.80 | terribly_new               |   6.84 |  1,701,929 |     69 | terribly     |
| **NEGany~remotely_close**             |       411,329 |        16,426 |      624.42 |   0.43 |            1.23 |  1,597 |     69.58 |  3,173,660 |   1.00 |   2,243.22 | close          |    3.65 | NEGATED    |    0.39 | remotely_close             |  23.70 | 72,839,589 |    694 | remotely     |
| **NEGmir~that_great**                 |         5,568 |         5,494 |      227.38 |   0.84 |            1.39 |    342 |     58.62 |    291,732 |   0.69 |     725.16 | great          |    3.57 | NEGMIR     |    0.66 | that_great                 |  13.45 |  1,701,929 |    286 | that         |
| **NEGany~ever_enough**                |       152,020 |       114,075 |      157.88 |   0.50 |            1.34 |    347 |     15.12 |  3,173,660 |   1.06 |     618.62 | enough         |    3.54 | NEGATED    |    0.45 | ever_enough                |  12.00 | 72,839,589 |    173 | ever         |
| **COM~ever_closer**                   |        61,475 |       114,075 |      273.08 |   1.00 |            2.06 |  6,307 |  6,031.92 | 69,662,736 |   0.02 |     538.66 | closer         |    3.51 | COMPLEMENT |    0.04 | ever_closer                |   3.44 | 72,839,589 |  6,305 | ever         |
| **NEGany~inherently_illegal**         |        30,194 |        47,803 |       54.99 |   0.64 |            1.59 |     92 |      4.01 |  3,173,660 |   1.17 |     252.59 | illegal        |    3.51 | NEGATED    |    0.60 | inherently_illegal         |   7.16 | 72,839,589 |     59 | inherently   |
| **NEGmir~terribly_original**          |         1,555 |         4,610 |       37.29 |   1.00 |            2.64 |     45 |      7.71 |    291,732 |   0.77 |     158.74 | original       |    3.40 | NEGMIR     |    0.83 | terribly_original          |   5.56 |  1,701,929 |     45 | terribly     |
| **NEGmir~exactly_clear**              |         6,722 |         1,041 |       42.92 |   0.98 |            2.23 |     53 |      9.08 |    291,732 |   0.76 |     173.89 | clear          |    3.38 | NEGMIR     |    0.81 | exactly_clear              |   5.95 |  1,701,929 |     52 | exactly      |
| **NEGmir~particularly_comfortable**   |         4,642 |        13,003 |       36.46 |   1.00 |            2.63 |     44 |      7.54 |    291,732 |   0.77 |     155.21 | comfortable    |    3.36 | NEGMIR     |    0.83 | particularly_comfortable   |   5.50 |  1,701,929 |     44 | particularly |
| **NEGany~ever_perfect**               |       104,659 |       114,075 |      194.69 |   0.44 |            1.24 |    489 |     21.31 |  3,173,660 |   1.01 |     706.71 | perfect        |    3.34 | NEGATED    |    0.40 | ever_perfect               |  13.25 | 72,839,589 |    216 | ever         |
| **NEGany~ever_sure**                  |       262,825 |       114,075 |       79.94 |   0.54 |            1.41 |    162 |      7.06 |  3,173,660 |   1.09 |     328.20 | sure           |    3.34 | NEGATED    |    0.49 | ever_sure                  |   8.57 | 72,839,589 |     87 | ever         |
| **NEGmir~remotely_close**             |        13,874 |         2,341 |      172.23 |   0.82 |            1.33 |    267 |     45.77 |    291,732 |   0.68 |     532.96 | close          |    3.28 | NEGMIR     |    0.65 | remotely_close             |  11.67 |  1,701,929 |    218 | remotely     |
| **NEGmir~any_different**              |        36,166 |         1,197 |       39.60 |   0.98 |            2.19 |     49 |      8.40 |    291,732 |   0.76 |     159.93 | different      |    3.24 | NEGMIR     |    0.81 | any_different              |   5.72 |  1,701,929 |     48 | any          |
| **NEGany~immediately_intuitive**      |        20,664 |        96,973 |       50.04 |   0.59 |            1.50 |     91 |      3.96 |  3,173,660 |   1.13 |     218.74 | intuitive      |    3.21 | NEGATED    |    0.55 | immediately_intuitive      |   6.81 | 72,839,589 |     54 | immediately  |
| **NEGmir~that_big**                   |         8,177 |         5,494 |       90.54 |   0.86 |            1.47 |    131 |     22.46 |    291,732 |   0.70 |     300.54 | big            |    3.17 | NEGMIR     |    0.69 | that_big                   |   8.52 |  1,701,929 |    113 | that         |
| **NEGmir~that_popular**               |         5,668 |         5,494 |       53.00 |   0.93 |            1.76 |     70 |     12.00 |    291,732 |   0.73 |     195.15 | popular        |    3.15 | NEGMIR     |    0.76 | that_popular               |   6.57 |  1,701,929 |     65 | that         |
| **NEGmir~ever_sick**                  |         1,895 |         5,060 |       32.31 |   1.00 |            2.58 |     39 |      6.69 |    291,732 |   0.77 |     137.57 | sick           |    3.15 | NEGMIR     |    0.83 | ever_sick                  |   5.17 |  1,701,929 |     39 | ever         |
| **NEGmir~particularly_revolutionary** |           485 |        13,003 |       32.31 |   1.00 |            2.58 |     39 |      6.69 |    291,732 |   0.77 |     137.57 | revolutionary  |    3.15 | NEGMIR     |    0.83 | particularly_revolutionary |   5.17 |  1,701,929 |     39 | particularly |
| **NEGany~remotely_qualified**         |        74,643 |        16,426 |       52.60 |   0.56 |            1.45 |    101 |      4.40 |  3,173,660 |   1.11 |     222.79 | qualified      |    3.13 | NEGATED    |    0.52 | remotely_qualified         |   6.97 | 72,839,589 |     57 | remotely     |
| **NEGmir~remotely_true**              |         6,191 |         2,341 |       49.69 |   0.92 |            1.73 |     66 |     11.31 |    291,732 |   0.73 |     181.65 | true           |    3.04 | NEGMIR     |    0.75 | remotely_true              |   6.36 |  1,701,929 |     61 | remotely     |
| **NEGmir~any_easier**                 |         2,386 |         1,197 |       49.69 |   0.92 |            1.73 |     66 |     11.31 |    291,732 |   0.73 |     181.65 | easier         |    3.04 | NEGMIR     |    0.75 | any_easier                 |   6.36 |  1,701,929 |     61 | any          |
| **NEGany~remotely_comparable**        |        12,252 |        16,426 |      105.93 |   0.43 |            1.21 |    277 |     12.07 |  3,173,660 |   0.99 |     375.73 | comparable     |    2.98 | NEGATED    |    0.38 | remotely_comparable        |   9.75 | 72,839,589 |    118 | remotely     |
| **NEGmir~necessarily_bad**            |        10,261 |         1,107 |       40.92 |   0.94 |            1.84 |     53 |      9.08 |    291,732 |   0.74 |     154.45 | bad            |    2.95 | NEGMIR     |    0.77 | necessarily_bad            |   5.79 |  1,701,929 |     50 | necessarily  |
| **NEGmir~inherently_bad**             |        10,261 |         5,133 |      115.77 |   0.79 |            1.25 |    188 |     32.23 |    291,732 |   0.66 |     342.53 | bad            |    2.86 | NEGMIR     |    0.62 | inherently_bad             |   9.52 |  1,701,929 |    148 | inherently   |
| **NEGany~inherently_evil**            |        22,706 |        47,803 |      309.07 |   0.32 |            1.01 |  1,123 |     48.93 |  3,173,660 |   0.86 |     905.80 | evil           |    2.81 | NEGATED    |    0.28 | inherently_evil            |  16.33 | 72,839,589 |    358 | inherently   |
| **NEGany~remotely_interested**        |       264,528 |        16,426 |      283.73 |   0.31 |            1.00 |  1,062 |     46.27 |  3,173,660 |   0.85 |     817.06 | interested     |    2.74 | NEGATED    |    0.27 | remotely_interested        |  15.62 | 72,839,589 |    330 | remotely     |
| **NEGmir~any_worse**                  |         8,790 |         1,197 |       69.00 |   0.83 |            1.36 |    105 |     18.00 |    291,732 |   0.68 |     217.46 | worse          |    2.73 | NEGMIR     |    0.66 | any_worse                  |   7.40 |  1,701,929 |     87 | any          |
| **NEGmir~particularly_radical**       |         1,072 |        13,003 |       25.69 |   1.00 |            2.48 |     31 |      5.31 |    291,732 |   0.77 |     109.35 | radical        |    2.73 | NEGMIR     |    0.83 | particularly_radical       |   4.61 |  1,701,929 |     31 | particularly |
| **NEGmir~that_keen**                  |         1,360 |         5,494 |       25.69 |   1.00 |            2.48 |     31 |      5.31 |    291,732 |   0.77 |     109.35 | keen           |    2.73 | NEGMIR     |    0.83 | that_keen                  |   4.61 |  1,701,929 |     31 | that         |
| **NEGmir~any_bigger**                 |         3,923 |         1,197 |       29.66 |   0.97 |            2.07 |     37 |      6.34 |    291,732 |   0.75 |     118.17 | bigger         |    2.73 | NEGMIR     |    0.80 | any_bigger                 |   4.94 |  1,701,929 |     36 | any          |
| **NEGmir~remotely_comparable**        |           283 |         2,341 |       35.94 |   0.94 |            1.79 |     47 |      8.06 |    291,732 |   0.74 |     134.02 | comparable     |    2.72 | NEGMIR     |    0.76 | remotely_comparable        |   5.42 |  1,701,929 |     44 | remotely     |
| **NEGmir~necessarily_true**           |         6,191 |         1,107 |       42.89 |   0.90 |            1.60 |     59 |     10.11 |    291,732 |   0.72 |     150.42 | true           |    2.69 | NEGMIR     |    0.73 | necessarily_true           |   5.89 |  1,701,929 |     53 | necessarily  |
| **NEGany~ever_satisfied**             |        62,862 |       114,075 |       57.42 |   0.42 |            1.21 |    151 |      6.58 |  3,173,660 |   0.99 |     203.01 | satisfied      |    2.57 | NEGATED    |    0.38 | ever_satisfied             |   7.18 | 72,839,589 |     64 | ever         |
| **NEGmir~yet_available**              |        10,284 |           815 |       23.20 |   1.00 |            2.44 |     28 |      4.80 |    291,732 |   0.77 |      98.77 | available      |    2.54 | NEGMIR     |    0.83 | yet_available              |   4.38 |  1,701,929 |     28 | yet          |
| **NEGany~inherently_negative**        |        53,385 |        47,803 |       66.59 |   0.39 |            1.15 |    193 |      8.41 |  3,173,660 |   0.95 |     222.63 | negative       |    2.50 | NEGATED    |    0.35 | inherently_negative        |   7.69 | 72,839,589 |     75 | inherently   |
| **NEGmir~terribly_interesting**       |        12,447 |         4,610 |       44.69 |   0.85 |            1.42 |     66 |     11.31 |    291,732 |   0.69 |     145.16 | interesting    |    2.44 | NEGMIR     |    0.68 | terribly_interesting       |   5.97 |  1,701,929 |     56 | terribly     |
| **NEGmir~particularly_fast**          |         1,259 |        13,003 |       21.54 |   1.00 |            2.41 |     26 |      4.46 |    291,732 |   0.77 |      91.71 | fast           |    2.39 | NEGMIR     |    0.83 | particularly_fast          |   4.22 |  1,701,929 |     26 | particularly |
| **NEGmir~terribly_interested**        |         8,255 |         4,610 |       31.63 |   0.91 |            1.63 |     43 |      7.37 |    291,732 |   0.72 |     112.46 | interested     |    2.36 | NEGMIR     |    0.74 | terribly_interested        |   5.06 |  1,701,929 |     39 | terribly     |
| **NEGmir~any_closer**                 |           993 |         1,197 |       45.17 |   0.83 |            1.35 |     69 |     11.83 |    291,732 |   0.68 |     141.82 | closer         |    2.33 | NEGMIR     |    0.65 | any_closer                 |   5.98 |  1,701,929 |     57 | any          |
| **NEGmir~exactly_new**                |        12,836 |         1,041 |       23.86 |   0.97 |            1.98 |     30 |      5.14 |    291,732 |   0.75 |      93.90 | new            |    2.31 | NEGMIR     |    0.80 | exactly_new                |   4.43 |  1,701,929 |     29 | exactly      |
| **NEGmir~terribly_unusual**           |         2,302 |         4,610 |       19.89 |   1.00 |            2.37 |     24 |      4.11 |    291,732 |   0.77 |      84.66 | unusual        |    2.23 | NEGMIR     |    0.83 | terribly_unusual           |   4.06 |  1,701,929 |     24 | terribly     |
| **NEGmir~terribly_special**           |        15,541 |         4,610 |       19.89 |   1.00 |            2.37 |     24 |      4.11 |    291,732 |   0.77 |      84.66 | special        |    2.23 | NEGMIR     |    0.83 | terribly_special           |   4.06 |  1,701,929 |     24 | terribly     |
| **NEGany~inherently_good**            |     1,681,795 |        47,803 |      231.72 |   0.24 |            0.84 |  1,177 |     51.28 |  3,173,660 |   0.74 |     554.72 | good           |    2.21 | NEGATED    |    0.20 | inherently_good            |  13.77 | 72,839,589 |    283 | inherently   |
| **NEGany~remotely_similar**           |       203,453 |        16,426 |      127.64 |   0.27 |            0.91 |    559 |     24.36 |  3,173,660 |   0.80 |     334.61 | similar        |    2.20 | NEGATED    |    0.23 | remotely_similar           |  10.35 | 72,839,589 |    152 | remotely     |
| **NEGmir~necessarily_right**          |         5,576 |         1,107 |       19.06 |   1.00 |            2.36 |     23 |      3.94 |    291,732 |   0.77 |      81.13 | right          |    2.15 | NEGMIR     |    0.83 | necessarily_right          |   3.97 |  1,701,929 |     23 | necessarily  |
| **NEGmir~that_impressive**            |         5,007 |         5,494 |       19.06 |   1.00 |            2.36 |     23 |      3.94 |    291,732 |   0.77 |      81.13 | impressive     |    2.15 | NEGMIR     |    0.83 | that_impressive            |   3.97 |  1,701,929 |     23 | that         |
| **NEGmir~any_good**                   |        31,585 |         1,197 |       26.00 |   0.91 |            1.65 |     35 |      6.00 |    291,732 |   0.73 |      93.53 | good           |    2.12 | NEGMIR     |    0.74 | any_good                   |   4.60 |  1,701,929 |     32 | any          |
| **NEGany~remotely_accurate**          |       152,299 |        16,426 |       44.16 |   0.37 |            1.12 |    134 |      5.84 |  3,173,660 |   0.93 |     143.78 | accurate       |    2.10 | NEGATED    |    0.33 | remotely_accurate          |   6.25 | 72,839,589 |     50 | remotely     |
| **NEGmir~inherently_illegal**         |           937 |         5,133 |       21.37 |   0.96 |            1.93 |     27 |      4.63 |    291,732 |   0.75 |      83.54 | illegal        |    2.10 | NEGMIR     |    0.79 | inherently_illegal         |   4.19 |  1,701,929 |     26 | inherently   |
| **NEGmir~remotely_believable**        |           600 |         2,341 |       17.40 |   1.00 |            2.32 |     21 |      3.60 |    291,732 |   0.77 |      74.08 | believable     |    1.96 | NEGMIR     |    0.83 | remotely_believable        |   3.80 |  1,701,929 |     21 | remotely     |
| **NEGmir~yet_certain**                |         1,800 |           815 |       17.40 |   1.00 |            2.32 |     21 |      3.60 |    291,732 |   0.77 |      74.08 | certain        |    1.96 | NEGMIR     |    0.83 | yet_certain                |   3.80 |  1,701,929 |     21 | yet          |
| **NEGmir~any_younger**                |           939 |         1,197 |       16.57 |   1.00 |            2.30 |     20 |      3.43 |    291,732 |   0.77 |      70.55 | younger        |    1.86 | NEGMIR     |    0.83 | any_younger                |   3.71 |  1,701,929 |     20 | any          |
| **NEGmir~exactly_easy**               |        18,610 |         1,041 |       16.57 |   1.00 |            2.30 |     20 |      3.43 |    291,732 |   0.77 |      70.55 | easy           |    1.86 | NEGMIR     |    0.83 | exactly_easy               |   3.71 |  1,701,929 |     20 | exactly      |
| **NEGmir~remotely_funny**             |         5,365 |         2,341 |       32.26 |   0.80 |            1.28 |     51 |      8.74 |    291,732 |   0.67 |      97.91 | funny          |    1.85 | NEGMIR     |    0.63 | remotely_funny             |   5.04 |  1,701,929 |     41 | remotely     |
| **NEGmir~immediately_available**      |        10,284 |         1,195 |      114.86 |   0.59 |            0.84 |    275 |     47.14 |    291,732 |   0.54 |     241.53 | available      |    1.85 | NEGMIR     |    0.42 | immediately_available      |   9.02 |  1,701,929 |    162 | immediately  |
| **NEGmir~that_fond**                  |         1,115 |         5,494 |       18.89 |   0.96 |            1.88 |     24 |      4.11 |    291,732 |   0.75 |      73.19 | fond           |    1.84 | NEGMIR     |    0.79 | that_fond                  |   3.94 |  1,701,929 |     23 | that         |
| **NEGmir~that_comfortable**           |         4,642 |         5,494 |       18.89 |   0.96 |            1.88 |     24 |      4.11 |    291,732 |   0.75 |      73.19 | comfortable    |    1.84 | NEGMIR     |    0.79 | that_comfortable           |   3.94 |  1,701,929 |     23 | that         |
| **NEGmir~necessarily_new**            |        12,836 |         1,107 |       18.89 |   0.96 |            1.88 |     24 |      4.11 |    291,732 |   0.75 |      73.19 | new            |    1.84 | NEGMIR     |    0.79 | necessarily_new            |   3.94 |  1,701,929 |     23 | necessarily  |
| **NEGmir~remotely_interesting**       |        12,447 |         2,341 |       42.80 |   0.73 |            1.10 |     77 |     13.20 |    291,732 |   0.63 |     115.20 | interesting    |    1.80 | NEGMIR     |    0.56 | remotely_interesting       |   5.72 |  1,701,929 |     56 | remotely     |
| **NEGmir~terribly_popular**           |         5,668 |         4,610 |       15.74 |   1.00 |            2.28 |     19 |      3.26 |    291,732 |   0.77 |      67.02 | popular        |    1.75 | NEGMIR     |    0.83 | terribly_popular           |   3.61 |  1,701,929 |     19 | terribly     |
| **NEGmir~yet_clear**                  |         6,722 |           815 |       15.74 |   1.00 |            2.28 |     19 |      3.26 |    291,732 |   0.77 |      67.02 | clear          |    1.75 | NEGMIR     |    0.83 | yet_clear                  |   3.61 |  1,701,929 |     19 | yet          |
| **NEGmir~yet_sure**                   |         6,761 |           815 |       15.74 |   1.00 |            2.28 |     19 |      3.26 |    291,732 |   0.77 |      67.02 | sure           |    1.75 | NEGMIR     |    0.83 | yet_sure                   |   3.61 |  1,701,929 |     19 | yet          |
| **NEGmir~remotely_similar**           |         7,011 |         2,341 |       52.66 |   0.66 |            0.98 |    107 |     18.34 |    291,732 |   0.59 |     127.32 | similar        |    1.70 | NEGMIR     |    0.49 | remotely_similar           |   6.25 |  1,701,929 |     71 | remotely     |
| **NEGmir~inherently_better**          |        14,013 |         5,133 |       33.89 |   0.75 |            1.14 |     59 |     10.11 |    291,732 |   0.64 |      93.95 | better         |    1.65 | NEGMIR     |    0.57 | inherently_better          |   5.11 |  1,701,929 |     44 | inherently   |
| **NEGmir~necessarily_better**         |        14,013 |         1,107 |       21.69 |   0.87 |            1.47 |     31 |      5.31 |    291,732 |   0.71 |      72.90 | better         |    1.63 | NEGMIR     |    0.70 | necessarily_better         |   4.17 |  1,701,929 |     27 | necessarily  |
| **NEGmir~inherently_improper**        |           142 |         5,133 |       14.91 |   1.00 |            2.25 |     18 |      3.09 |    291,732 |   0.77 |      63.49 | improper       |    1.63 | NEGMIR     |    0.83 | inherently_improper        |   3.52 |  1,701,929 |     18 | inherently   |
| **NEGmir~yet_ready**                  |         3,034 |           815 |       14.91 |   1.00 |            2.25 |     18 |      3.09 |    291,732 |   0.77 |      63.49 | ready          |    1.63 | NEGMIR     |    0.83 | yet_ready                  |   3.52 |  1,701,929 |     18 | yet          |
| **NEGmir~any_clearer**                |           130 |         1,197 |       14.09 |   1.00 |            2.23 |     17 |      2.91 |    291,732 |   0.77 |      59.97 | clearer        |    1.50 | NEGMIR     |    0.83 | any_clearer                |   3.42 |  1,701,929 |     17 | any          |
| **NEGmir~remotely_surprising**        |         2,662 |         2,341 |       14.09 |   1.00 |            2.23 |     17 |      2.91 |    291,732 |   0.77 |      59.97 | surprising     |    1.50 | NEGMIR     |    0.83 | remotely_surprising        |   3.42 |  1,701,929 |     17 | remotely     |
| **NEGmir~any_higher**                 |         2,893 |         1,197 |       17.06 |   0.91 |            1.62 |     23 |      3.94 |    291,732 |   0.73 |      61.24 | higher         |    1.42 | NEGMIR     |    0.74 | any_higher                 |   3.72 |  1,701,929 |     21 | any          |
| **NEGmir~remotely_possible**          |         3,160 |         2,341 |       29.09 |   0.73 |            1.11 |     52 |      8.91 |    291,732 |   0.63 |      78.73 | possible       |    1.42 | NEGMIR     |    0.56 | remotely_possible          |   4.72 |  1,701,929 |     38 | remotely     |
| **NEGmir~that_clear**                 |         6,722 |         5,494 |       14.74 |   0.95 |            1.78 |     19 |      3.26 |    291,732 |   0.74 |      56.03 | clear          |    1.30 | NEGMIR     |    0.78 | that_clear                 |   3.47 |  1,701,929 |     18 | that         |
| **NEGmir~necessarily_illegal**        |           937 |         1,107 |       12.43 |   1.00 |            2.18 |     15 |      2.57 |    291,732 |   0.77 |      52.91 | illegal        |    1.20 | NEGMIR     |    0.83 | necessarily_illegal        |   3.21 |  1,701,929 |     15 | necessarily  |
| **NEGmir~terribly_clear**             |         6,722 |         4,610 |       12.43 |   1.00 |            2.18 |     15 |      2.57 |    291,732 |   0.77 |      52.91 | clear          |    1.20 | NEGMIR     |    0.83 | terribly_clear             |   3.21 |  1,701,929 |     15 | terribly     |
| **NEGmir~inherently_evil**            |         1,271 |         5,133 |       41.03 |   0.59 |            0.83 |     99 |     16.97 |    291,732 |   0.53 |      85.70 | evil           |    1.18 | NEGMIR     |    0.41 | inherently_evil            |   5.39 |  1,701,929 |     58 | inherently   |
| **NEGany~inherently_problematic**     |        33,408 |        47,803 |       45.93 |   0.21 |            0.77 |    277 |     12.07 |  3,173,660 |   0.68 |      98.70 | problematic    |    1.17 | NEGATED    |    0.17 | inherently_problematic     |   6.03 | 72,839,589 |     58 | inherently   |
| **NEGmir~immediately_clear**          |         6,722 |         1,195 |       23.63 |   0.72 |            1.09 |     43 |      7.37 |    291,732 |   0.62 |      62.95 | clear          |    1.13 | NEGMIR     |    0.55 | immediately_clear          |   4.24 |  1,701,929 |     31 | immediately  |
| **NEGmir~terribly_remarkable**        |         3,238 |         4,610 |       11.60 |   1.00 |            2.15 |     14 |      2.40 |    291,732 |   0.77 |      49.38 | remarkable     |    1.03 | NEGMIR     |    0.83 | terribly_remarkable        |   3.10 |  1,701,929 |     14 | terribly     |
| **NEGmir~remotely_new**               |        12,836 |         2,341 |       15.23 |   0.86 |            1.43 |     22 |      3.77 |    291,732 |   0.70 |      50.62 | new            |    1.00 | NEGMIR     |    0.69 | remotely_new               |   3.49 |  1,701,929 |     19 | remotely     |



In [28]:
nb_show_table(NEG_bigrams_sample.l1.value_counts().to_frame('subtotal in selected bigrams'))


|                |   `subtotal in selected bigrams` |
|:---------------|---------------------------------:|
| **NEGATED**    |                              106 |
| **NEGMIR**     |                               84 |
| **COMPLEMENT** |                                1 |




|                |   `subtotal in selected bigrams` |
|:---------------|---------------------------------:|
| **NEGATED**    |                              106 |
| **NEGMIR**     |                               84 |
| **COMPLEMENT** |                                1 |



In [32]:
# nb_show_table(NEG_bigrams_sample.filter(like='O', axis=0).T)


|                 | `COM~ever_closer`    |
|:----------------|:---------------------|
| **adj_total**   | 61475                |
| **adv_total**   | 114075               |
| **unexp_f**     | 273.0758999999998    |
| **P1**          | 0.9996829032897949   |
| **odds_r_disc** | 2.0608150959014893   |
| **f2**          | 6307                 |
| **exp_f**       | 6031.9241            |
| **f1**          | 69662736             |
| **MI**          | 0.019229218363761902 |
| **G2**          | 538.66082            |
| **adj**         | closer               |
| **LRC**         | 3.5134613513946533   |
| **l1**          | COMPLEMENT           |
| **dP1**         | 0.043301016092300415 |
| **l2**          | ever_closer          |
| **t**           | 3.439067840576172    |
| **N**           | 72839589             |
| **f**           | 6305                 |
| **adv**         | ever                 |



```python
nb_show_table(NEG_bigrams_sample.filter(like='O', axis=0).T)
```
(_+ manual sort & round_)

|                 | `COM~ever_closer` |
|:----------------|------------------:|
| **N**           |        72,839,589 |
| **f**           |             6,305 |
| **l1**          |        COMPLEMENT |
| **l2**          |       ever_closer |
| **f1**          |        69,662,736 |
| **f2**          |             6,307 |
| **adv**         |              ever |
| **adj**         |            closer |
| **adv_total**   |           114,075 |
| **adj_total**   |            61,475 |
| **exp_f**       |             6,032 |
| **unexp_f**     |               273 |
| **P1**          |            0.9997 |
| **dP1**         |            0.0433 |
| **LRC**         |              3.51 |
| **G2**          |             538.6 |
| **odds_r_disc** |              2.06 |
| **MI**          |            0.0192 |
| **t**           |              3.44 |

